## DATA

In [ ]:
import pandas as pd
from plotting_utils import *
from plotting_ACs import *
from clustering_utils import *

### Amacrine cells

In [ ]:
df = pd.read_pickle('AC_data.pkl') 
nb_clusters = 25
cluster_id= df['cluster ID (diag)']

### Bipolar cells

In [ ]:
df_bc = pd.read_pickle('BC_data.pkl') 
bc_nb_clusters = 24
bc_cluster_id= df_bc['cluster ID (full)']

## SPECTRAL CONTRAST

### Amacrine cells

In [ ]:
spectral_contrast = np.zeros((df['roi'].shape[0], 3))

for i in range(df['roi'].shape[0]):
    spectral_contrast[i,0] = (abs(df['green_center amplitude'][i]) - abs(df['uv_center amplitude'][i])) / (abs(df['green_center amplitude'][i]) + abs(df['uv_center amplitude'][i]))
    spectral_contrast[i,1] = (abs(df['green_ring amplitude'][i]) - abs(df['uv_ring amplitude'][i])) / (abs(df['green_ring amplitude'][i]) + abs(df['uv_ring amplitude'][i])) 
    spectral_contrast[i,2] = (abs(df['green_surround amplitude'][i]) - abs(df['uv_surround amplitude'][i])) / (abs(df['green_surround amplitude'][i]) + abs(df['uv_surround amplitude'][i])) 

df['spectral_contrast_center'] = spectral_contrast[:,0]
df['spectral_contrast_ring'] = spectral_contrast[:,1]
df['spectral_contrast_surround'] = spectral_contrast[:,2]

### Bipolar cells

In [ ]:
bc_spectral_contrast = np.zeros((df_bc['roi'].shape[0], 2))

for i in range(df_bc['roi'].shape[0]):
    bc_spectral_contrast[i,0] = (abs(df_bc['green_center amplitude'][i]) - abs(df_bc['uv_center amplitude'][i])) / (abs(df_bc['green_center amplitude'][i]) + abs(df_bc['uv_center amplitude'][i]))
    bc_spectral_contrast[i,1] = (abs(df_bc['green_surround amplitude'][i]) - abs(df_bc['uv_surround amplitude'][i])) / (abs(df_bc['green_surround amplitude'][i]) + abs(df_bc['uv_surround amplitude'][i])) 

df_bc['spectral_contrast_center'] = bc_spectral_contrast[:,0]
df_bc['spectral_contrast_surround'] = bc_spectral_contrast[:,1]

## ON-OFF INDEX

### Amacrine cells

In [ ]:
line_duration= 1.6 #in ms
baseline_time_ms = int(1000/line_duration)
on_time_ms = int(1000/line_duration) # in 1 s how many data points we have 
off_time_ms = int(1000/line_duration)

a=np.zeros(baseline_time_ms)
b=np.ones(on_time_ms)
c=np.zeros(off_time_ms)

stimulus= np.concatenate((a,b,c))

In [ ]:
convolved_response_c = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
convolved_response_r = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
convolved_response_s = np.zeros((df['roi'].shape[0],stimulus.shape[0]))

line_duration_s = 0.0016
kernel_length_line = np.int(np.floor(2/line_duration_s))
offset_after = np.int(np.floor(kernel_length_line*.25)) #lines to include into the future (using 1/4 of kernel length)
offset_before = kernel_length_line-offset_after
kernel_past = offset_before - 1
kernel_future = offset_after

for i in range(df['roi'].shape[0]):
    convolved_response_uv_c = np.convolve(stimulus, np.flip(df['uv_center'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_green_c = np.convolve(stimulus, np.flip(df['green_center'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_uv_r = np.convolve(stimulus, np.flip(df['uv_ring'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_green_r = np.convolve(stimulus, np.flip(df['green_ring'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_uv_s = np.convolve(stimulus, np.flip(df['uv_surround'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_green_s = np.convolve(stimulus, np.flip(df['green_surround'][i]), mode='full')[kernel_future:-kernel_past]
    
    convolved_response_c_avg = (convolved_response_uv_c + convolved_response_green_c)/2
    convolved_response_r_avg = (convolved_response_uv_r + convolved_response_green_r)/2
    convolved_response_s_avg = (convolved_response_uv_s + convolved_response_green_s)/2
    
    convolved_response_c[i,:] = convolved_response_c_avg-convolved_response_c_avg.min()
    convolved_response_r[i,:] = convolved_response_r_avg-convolved_response_r_avg.min()
    convolved_response_s[i,:] = convolved_response_s_avg-convolved_response_s_avg.min()

In [ ]:
window_on_start = 625
window_on_end = 1250
window_off_start = 1250
window_off_end = 1875

ooindex = np.zeros((df['roi'].shape[0],3))

for i in range(df['roi'].shape[0]):
    ooindex[i,0] = (convolved_response_c[i,window_on_start:window_on_end].mean() - convolved_response_c[i,window_off_start:window_off_end].mean())/(convolved_response_c[i,window_on_start:window_on_end].mean() + convolved_response_c[i,window_off_start:window_off_end].mean())
    ooindex[i,1] = (convolved_response_r[i,window_on_start:window_on_end].mean() - convolved_response_r[i,window_off_start:window_off_end].mean())/(convolved_response_r[i,window_on_start:window_on_end].mean() + convolved_response_r[i,window_off_start:window_off_end].mean())
    ooindex[i,2] = (convolved_response_s[i,window_on_start:window_on_end].mean() - convolved_response_s[i,window_off_start:window_off_end].mean())/(convolved_response_s[i,window_on_start:window_on_end].mean() + convolved_response_s[i,window_off_start:window_off_end].mean())

df['OOi_center'] = ooindex[:,0]
df['OOi_ring'] = ooindex[:,1]
df['OOi_surround'] = ooindex[:,2]

### Bipolar cells

In [ ]:
line_duration= 1.6 #in ms
baseline_time_ms = int(1000/line_duration)
on_time_ms = int(1000/line_duration) # in 1 s how many data points we have
off_time_ms = int(1000/line_duration)

a=np.zeros(baseline_time_ms)
b=np.ones(on_time_ms)
c=np.zeros(off_time_ms)

stimulus= np.concatenate((a,b,c))

In [ ]:
bc_convolved_response_c = np.zeros((df_bc['roi'].shape[0],stimulus.shape[0]))
bc_convolved_response_s = np.zeros((df_bc['roi'].shape[0],stimulus.shape[0]))

line_duration_s = 0.0016
kernel_length_line = np.int(np.floor(2/line_duration_s))
offset_after = np.int(np.floor(kernel_length_line*.25)) #lines to include into the future (using 1/4 of kernel length)
offset_before = kernel_length_line-offset_after
kernel_past = offset_before - 1
kernel_future = offset_after

for i in range(df_bc['roi'].shape[0]):
    bc_convolved_response_uv_c = np.convolve(stimulus, np.flip(df_bc['uv_center'][i]), mode='full')[kernel_future:-kernel_past]
    bc_convolved_response_green_c = np.convolve(stimulus, np.flip(df_bc['green_center'][i]), mode='full')[kernel_future:-kernel_past]
    bc_convolved_response_uv_s = np.convolve(stimulus, np.flip(df_bc['uv_surround'][i]), mode='full')[kernel_future:-kernel_past]
    bc_convolved_response_green_s = np.convolve(stimulus, np.flip(df_bc['green_surround'][i]), mode='full')[kernel_future:-kernel_past]
    
    bc_convolved_response_c_avg = (bc_convolved_response_uv_c + bc_convolved_response_green_c)/2
    bc_convolved_response_s_avg = (bc_convolved_response_uv_s + bc_convolved_response_green_s)/2
    
    bc_convolved_response_c[i,:] = bc_convolved_response_c_avg - bc_convolved_response_c_avg.min()
    bc_convolved_response_s[i,:] = bc_convolved_response_s_avg - bc_convolved_response_s_avg.min()

In [ ]:
window_on_start = 625
window_on_end = 1250

window_off_start = 1250
window_off_end = 1875

bc_ooindex = np.zeros((df_bc['roi'].shape[0],2))

for i in range(df_bc['roi'].shape[0]):
    bc_ooindex[i,0] = (bc_convolved_response_c[i,window_on_start:window_on_end].mean() - bc_convolved_response_c[i,window_off_start:window_off_end].mean())/(bc_convolved_response_c[i,window_on_start:window_on_end].mean() + bc_convolved_response_c[i,window_off_start:window_off_end].mean())
    bc_ooindex[i,1] = (bc_convolved_response_s[i,window_on_start:window_on_end].mean() - bc_convolved_response_s[i,window_off_start:window_off_end].mean())/(bc_convolved_response_s[i,window_on_start:window_on_end].mean() + bc_convolved_response_s[i,window_off_start:window_off_end].mean())

df_bc['OOi_center'] = bc_ooindex[:,0]
df_bc['OOi_surround'] = bc_ooindex[:,1]

## Example fields - Figure 1

In [ ]:
#field 1 (ventral)
import datetime
one_date = datetime.date(2022, 4, 13) 
one_exp = 1
one_field = 1
           
first_field = df[(df['date'] == one_date) & 
                (df['exp_num'] == one_exp) &
                (df['field_id'] == one_field)]

first_field = first_field.reset_index(drop = True)

In [ ]:
#field 2 (dorsal)
two_date = datetime.date(2022, 5, 18) 
two_exp = 4
two_field = 1
           
second_field = df[(df['date'] == two_date) & 
                (df['exp_num'] == two_exp) &
                (df['field_id'] == two_field)]

second_field = second_field.reset_index(drop = True)

In [ ]:
# Calculate amplitudes for plotting
nb_rois= 6
roi_contrast_center= np.zeros(((nb_rois),2))
roi_contrast_ring= np.zeros(((nb_rois),2))
roi_contrast_surround= np.zeros(((nb_rois),2))

roi_contrast_center[0,0]= first_field['green_center amplitude'].loc[first_field['roi'] == 14].values[0]
roi_contrast_center[0,1]= first_field['uv_center amplitude'].loc[first_field['roi'] == 14].values[0]
roi_contrast_ring[0,0]= first_field['green_ring amplitude'].loc[first_field['roi'] == 14].values[0]
roi_contrast_ring[0,1]= first_field['uv_ring amplitude'].loc[first_field['roi'] == 14].values[0]
roi_contrast_surround[0,0]= first_field['green_surround amplitude'].loc[first_field['roi'] == 14].values[0]
roi_contrast_surround[0,1]= first_field['uv_surround amplitude'].loc[first_field['roi'] == 14].values[0]

roi_contrast_center[1,0]= first_field['green_center amplitude'].loc[first_field['roi'] == 23].values[0]
roi_contrast_center[1,1]= first_field['uv_center amplitude'].loc[first_field['roi'] == 23].values[0]
roi_contrast_ring[1,0]= first_field['green_ring amplitude'].loc[first_field['roi'] == 23].values[0]
roi_contrast_ring[1,1]= first_field['uv_ring amplitude'].loc[first_field['roi'] == 23].values[0]
roi_contrast_surround[1,0]= first_field['green_surround amplitude'].loc[first_field['roi'] == 23].values[0]
roi_contrast_surround[1,1]= first_field['uv_surround amplitude'].loc[first_field['roi'] == 23].values[0]

roi_contrast_center[2,0]= first_field['green_center amplitude'].loc[first_field['roi'] == 13].values[0]
roi_contrast_center[2,1]= first_field['uv_center amplitude'].loc[first_field['roi'] == 13].values[0]
roi_contrast_ring[2,0]= first_field['green_ring amplitude'].loc[first_field['roi'] == 13].values[0]
roi_contrast_ring[2,1]= first_field['uv_ring amplitude'].loc[first_field['roi'] == 13].values[0]
roi_contrast_surround[2,0]= first_field['green_surround amplitude'].loc[first_field['roi'] == 13].values[0]
roi_contrast_surround[2,1]= first_field['uv_surround amplitude'].loc[first_field['roi'] == 13].values[0]

roi_contrast_center[3,0]= second_field['green_center amplitude'].loc[second_field['roi'] == 8].values[0] 
roi_contrast_center[3,1]= second_field['uv_center amplitude'].loc[second_field['roi'] == 8].values[0]
roi_contrast_ring[3,0]= second_field['green_ring amplitude'].loc[second_field['roi'] == 8].values[0]
roi_contrast_ring[3,1]= second_field['uv_ring amplitude'].loc[second_field['roi'] == 8].values[0]
roi_contrast_surround[3,0]= second_field['green_surround amplitude'].loc[second_field['roi'] == 8].values[0]
roi_contrast_surround[3,1]= second_field['uv_surround amplitude'].loc[second_field['roi'] == 8].values[0]

roi_contrast_center[4,0]= second_field['green_center amplitude'].loc[second_field['roi'] == 5].values[0] 
roi_contrast_center[4,1]= second_field['uv_center amplitude'].loc[second_field['roi'] == 5].values[0]
roi_contrast_ring[4,0]= second_field['green_ring amplitude'].loc[second_field['roi'] == 5].values[0]
roi_contrast_ring[4,1]= second_field['uv_ring amplitude'].loc[second_field['roi'] == 5].values[0]
roi_contrast_surround[4,0]= second_field['green_surround amplitude'].loc[second_field['roi'] == 5].values[0]
roi_contrast_surround[4,1]= second_field['uv_surround amplitude'].loc[second_field['roi'] == 5].values[0]

roi_contrast_center[5,0]= second_field['green_center amplitude'].loc[second_field['roi'] == 17].values[0] 
roi_contrast_center[5,1]= second_field['uv_center amplitude'].loc[second_field['roi'] == 17].values[0]
roi_contrast_ring[5,0]= second_field['green_ring amplitude'].loc[second_field['roi'] == 17].values[0]
roi_contrast_ring[5,1]= second_field['uv_ring amplitude'].loc[second_field['roi'] == 17].values[0]
roi_contrast_surround[5,0]= second_field['green_surround amplitude'].loc[second_field['roi'] == 17].values[0]
roi_contrast_surround[5,1]= second_field['uv_surround amplitude'].loc[second_field['roi'] == 17].values[0]

In [ ]:
# PLOT ROI KERNELS
roi_1 = first_field[first_field['roi'] == 14]
roi_2 = first_field[first_field['roi'] == 23] 
roi_3 = first_field[first_field['roi'] == 13]
roi_4 = second_field[second_field['roi'] == 8]
roi_5 = second_field[second_field['roi'] == 5]
roi_6 = second_field[second_field['roi'] == 17]

fig1, ax = plt.subplots(6,6, figsize=(15,15),sharey = 'row')
bar_colors = ['purple', 'darkgreen']

ax[0,0].plot(roi_1['uv_center'].values[0], 'purple')
ax[0,0].plot(roi_1['green_center'].values[0], 'darkgreen') 
ax[0,1].bar([0,1],[roi_contrast_center[0,1], roi_contrast_center[0,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[0,2].plot(roi_1['uv_ring'].values[0], 'purple') 
ax[0,2].plot(roi_1['green_ring'].values[0], 'darkgreen') 
ax[0,3].bar([0,1],[roi_contrast_ring[0,1], roi_contrast_ring[0,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[0,4].plot(roi_1['uv_surround'].values[0],'purple') 
ax[0,4].plot(roi_1['green_surround'].values[0], 'darkgreen') 
ax[0,5].bar([0,1],[roi_contrast_surround[0,1], roi_contrast_surround[0,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[0,0].vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1

ax[1,0].plot(roi_2['uv_center'].values[0], 'purple')
ax[1,0].plot(roi_2['green_center'].values[0], 'darkgreen') 
ax[1,1].bar([0,1],[roi_contrast_center[1,1], roi_contrast_center[1,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[1,2].plot(roi_2['uv_ring'].values[0], 'purple') 
ax[1,2].plot(roi_2['green_ring'].values[0], 'darkgreen') 
ax[1,3].bar([0,1],[roi_contrast_ring[1,1], roi_contrast_ring[1,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[1,4].plot(roi_2['uv_surround'].values[0],'purple') 
ax[1,4].plot(roi_2['green_surround'].values[0], 'darkgreen') 
ax[1,5].bar([0,1],[roi_contrast_surround[1,1], roi_contrast_surround[1,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[1,0].vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1

ax[2,0].plot(roi_3['uv_center'].values[0], 'purple')
ax[2,0].plot(roi_3['green_center'].values[0], 'darkgreen') 
ax[2,1].bar([0,1],[roi_contrast_center[2,1], roi_contrast_center[2,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[2,2].plot(roi_3['uv_ring'].values[0], 'purple') 
ax[2,2].plot(roi_3['green_ring'].values[0], 'darkgreen') 
ax[2,3].bar([0,1],[roi_contrast_ring[2,1], roi_contrast_ring[2,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[2,4].plot(roi_3['uv_surround'].values[0],'purple') 
ax[2,4].plot(roi_3['green_surround'].values[0], 'darkgreen') 
ax[2,5].bar([0,1],[roi_contrast_surround[2,1], roi_contrast_surround[2,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[2,0].vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1

ax[3,0].plot(roi_4['uv_center'].values[0], 'purple')
ax[3,0].plot(roi_4['green_center'].values[0], 'darkgreen') 
ax[3,1].bar([0,1],[roi_contrast_center[3,1], roi_contrast_center[3,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[3,2].plot(roi_4['uv_ring'].values[0], 'purple') 
ax[3,2].plot(roi_4['green_ring'].values[0], 'darkgreen') 
ax[3,3].bar([0,1],[roi_contrast_ring[3,1], roi_contrast_ring[3,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[3,4].plot(roi_4['uv_surround'].values[0],'purple') 
ax[3,4].plot(roi_4['green_surround'].values[0], 'darkgreen') 
ax[3,5].bar([0,1],[roi_contrast_surround[3,1], roi_contrast_surround[3,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[3,0].vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1

ax[4,0].plot(roi_5['uv_center'].values[0], 'purple')
ax[4,0].plot(roi_5['green_center'].values[0], 'darkgreen') 
ax[4,1].bar([0,1],[roi_contrast_center[4,1], roi_contrast_center[4,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[4,2].plot(roi_5['uv_ring'].values[0], 'purple') 
ax[4,2].plot(roi_5['green_ring'].values[0], 'darkgreen') 
ax[4,3].bar([0,1],[roi_contrast_ring[4,1], roi_contrast_ring[4,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[4,4].plot(roi_5['uv_surround'].values[0],'purple') 
ax[4,4].plot(roi_5['green_surround'].values[0], 'darkgreen') 
ax[4,5].bar([0,1],[roi_contrast_surround[4,1], roi_contrast_surround[4,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[4,0].vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1

ax[5,0].plot(roi_6['uv_center'].values[0], 'purple')
ax[5,0].plot(roi_6['green_center'].values[0], 'darkgreen') 
ax[5,1].bar([0,1],[roi_contrast_center[5,1], roi_contrast_center[5,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[5,2].plot(roi_6['uv_ring'].values[0], 'purple') 
ax[5,2].plot(roi_6['green_ring'].values[0], 'darkgreen') 
ax[5,3].bar([0,1],[roi_contrast_ring[5,1], roi_contrast_ring[5,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[5,4].plot(roi_6['uv_surround'].values[0],'purple') 
ax[5,4].plot(roi_6['green_surround'].values[0], 'darkgreen') 
ax[5,5].bar([0,1],[roi_contrast_surround[5,1], roi_contrast_surround[5,0]],align='center', capsize=2, width=1, color=bar_colors)
ax[5,0].vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1

ax[0,0].axvline(x= 938, color='grey', linestyle='--')
ax[0,2].axvline(x= 938, color='grey', linestyle='--')
ax[0,4].axvline(x= 938, color='grey', linestyle='--')
ax[0,0].axhline(y= 0, color='grey', linestyle='--')
ax[1,0].axhline(y= 0, color='grey', linestyle='--')
ax[2,0].axhline(y= 0, color='grey', linestyle='--')
ax[3,0].axhline(y= 0, color='grey', linestyle='--')
ax[4,0].axhline(y= 0, color='grey', linestyle='--')
ax[5,0].axhline(y= 0, color='grey', linestyle='--')

sns.despine()
plt.show()

In [ ]:
# SC CIRCLES
nb_rois = 6
roi_spectral_contrast = np.zeros(((nb_rois),3))

roi_spectral_contrast[0,0] = first_field['spectral_contrast_surround'].loc[first_field['roi'] == 14].values[0] 
roi_spectral_contrast[0,1] = first_field['spectral_contrast_ring'].loc[first_field['roi'] == 14].values[0]
roi_spectral_contrast[0,2] = first_field['spectral_contrast_center'].loc[first_field['roi'] == 14].values[0]

roi_spectral_contrast[1,0] = first_field['spectral_contrast_surround'].loc[first_field['roi'] == 23].values[0] 
roi_spectral_contrast[1,1] = first_field['spectral_contrast_ring'].loc[first_field['roi'] == 23].values[0]
roi_spectral_contrast[1,2] = first_field['spectral_contrast_center'].loc[first_field['roi'] == 23].values[0]

roi_spectral_contrast[2,0] = first_field['spectral_contrast_surround'].loc[first_field['roi'] == 13].values[0]
roi_spectral_contrast[2,1] = first_field['spectral_contrast_ring'].loc[first_field['roi'] == 13].values[0]
roi_spectral_contrast[2,2] = first_field['spectral_contrast_center'].loc[first_field['roi'] == 13].values[0]

roi_spectral_contrast[3,0] = second_field['spectral_contrast_surround'].loc[second_field['roi'] == 8].values[0]
roi_spectral_contrast[3,1] = second_field['spectral_contrast_ring'].loc[second_field['roi'] == 8].values[0]
roi_spectral_contrast[3,2] = second_field['spectral_contrast_center'].loc[second_field['roi'] == 8].values[0]

roi_spectral_contrast[4,0] = second_field['spectral_contrast_surround'].loc[second_field['roi'] == 5].values[0]
roi_spectral_contrast[4,1] = second_field['spectral_contrast_ring'].loc[second_field['roi'] == 5].values[0]
roi_spectral_contrast[4,2] = second_field['spectral_contrast_center'].loc[second_field['roi'] == 5].values[0]

roi_spectral_contrast[5,0] = second_field['spectral_contrast_surround'].loc[second_field['roi'] == 17].values[0]
roi_spectral_contrast[5,1] = second_field['spectral_contrast_ring'].loc[second_field['roi'] == 17].values[0]
roi_spectral_contrast[5,2] = second_field['spectral_contrast_center'].loc[second_field['roi'] == 17].values[0]

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('PiYG', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([1, 1, 1, 1])

# center1 = np.amax((first_field['spectral_contrast_center'].max(), np.abs(first_field['spectral_contrast_center'].min())))
# ring1 = np.amax((first_field['spectral_contrast_ring'].max(), np.abs(first_field['spectral_contrast_ring'].min())))
# surround1 = np.amax((first_field['spectral_contrast_surround'].max(), np.abs(first_field['spectral_contrast_surround'].min())))
# center2 = np.amax((second_field['spectral_contrast_center'].max(), np.abs(second_field['spectral_contrast_center'].min())))
# ring2 = np.amax((second_field['spectral_contrast_ring'].max(), np.abs(second_field['spectral_contrast_ring'].min())))
# surround2 = np.amax((second_field['spectral_contrast_surround'].max(), np.abs(second_field['spectral_contrast_surround'].min())))

limit = 0.5 #np.amax((center1, ring1, surround1, center2, ring2, surround2))

vmax = limit 
vmin = -limit
white_start = 1/4* limit  
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

def plot_colorCircles(color_pref, cmap, limit = None, colorbar = False, savefig = False, path = None):
    center = [0.5, 0.5]
    radii = [0.3, 0.2, 0.1]
    nb_rois = 6
    if limit is None:
        limit = np.amax((np.amax(color_pref), np.abs(np.amin(color_pref))))
    fig, ax = plt.subplots(nb_rois, 1, figsize=(1.3,nb_rois*1.5))
    for k in range(nb_rois):
        patches = []
        colors = color_pref[k,:]
        for r in radii: 
            circle = Circle((center), r)
            patches.append(circle)
        p = PatchCollection(patches, cmap = cmap, edgecolor = 'black', linewidth = 0.5)
        p.set_array(colors)
        p.set_clim([-limit, limit])
        ax[k].set_axis_off()
        ax[k].add_collection(p)
    if colorbar:
        fig.colorbar(p, ax=ax[-1])
    if savefig:
        plt.savefig(path, dpi = 600)
    plt.show() 

In [ ]:
plot_colorCircles(roi_spectral_contrast, newcmp, limit=0.5, colorbar = True)

In [ ]:
# OOi CIRCLES
nb_rois = 6
roi_ooindex = np.zeros(((nb_rois),3))

roi_ooindex[0,0] = first_field['OOi_surround'].loc[first_field['roi'] == 14].values[0]
roi_ooindex[0,1] = first_field['OOi_ring'].loc[first_field['roi'] == 14].values[0]
roi_ooindex[0,2] = first_field['OOi_center'].loc[first_field['roi'] == 14].values[0]

roi_ooindex[1,0] = first_field['OOi_surround'].loc[first_field['roi'] == 23].values[0]
roi_ooindex[1,1] = first_field['OOi_ring'].loc[first_field['roi'] == 23].values[0]
roi_ooindex[1,2] = first_field['OOi_center'].loc[first_field['roi'] == 23].values[0]

roi_ooindex[2,0] = first_field['OOi_surround'].loc[first_field['roi'] == 13].values[0]
roi_ooindex[2,1] = first_field['OOi_ring'].loc[first_field['roi'] == 13].values[0]
roi_ooindex[2,2] = first_field['OOi_center'].loc[first_field['roi'] == 13].values[0]

roi_ooindex[3,0] = second_field['OOi_surround'].loc[second_field['roi'] == 8].values[0]
roi_ooindex[3,1] = second_field['OOi_ring'].loc[second_field['roi'] == 8].values[0]
roi_ooindex[3,2] = second_field['OOi_center'].loc[second_field['roi'] == 8].values[0]

roi_ooindex[4,0] = second_field['OOi_surround'].loc[second_field['roi'] == 5].values[0]
roi_ooindex[4,1] = second_field['OOi_ring'].loc[second_field['roi'] == 5].values[0]
roi_ooindex[4,2] = second_field['OOi_center'].loc[second_field['roi'] == 5].values[0]

roi_ooindex[5,0] = second_field['OOi_surround'].loc[second_field['roi'] == 17].values[0]
roi_ooindex[5,1] = second_field['OOi_ring'].loc[second_field['roi'] == 17].values[0]
roi_ooindex[5,2] = second_field['OOi_center'].loc[second_field['roi'] == 17].values[0]

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('Greys_r', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([0.5, 0.5, 0.5, 0.5])

center1 = np.amax((first_field['OOi_center'].max(), np.abs(first_field['OOi_center'].min())))
ring1 = np.amax((first_field['OOi_ring'].max(), np.abs(first_field['OOi_ring'].min())))
surround1 = np.amax((first_field['OOi_surround'].max(), np.abs(first_field['OOi_surround'].min())))
center2 = np.amax((second_field['OOi_center'].max(), np.abs(second_field['OOi_center'].min())))
ring2 = np.amax((second_field['OOi_ring'].max(), np.abs(second_field['OOi_ring'].min())))
surround2 = np.amax((second_field['OOi_surround'].max(), np.abs(second_field['OOi_surround'].min())))

limit = np.amax((center1, ring1, surround1, center2, ring2, surround2))

vmax = limit 
vmin = -limit
white_start = 1/4* limit  
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

def plot_polarityCircles(color_pref, cmap, limit = None, colorbar = False, savefig = False, path = None):
    center = [0.5, 0.5]
    radii = [0.3, 0.2, 0.1]
    nb_rois = 6
    if limit is None:
        limit = np.amax((np.amax(color_pref), np.abs(np.amin(color_pref))))
    fig, ax = plt.subplots(nb_rois, 1, figsize=(1.3,nb_rois*1.5))
    for k in range(nb_rois):
        patches = []
        colors = color_pref[k,:]
        for r in radii: 
            circle = Circle((center), r)
            patches.append(circle)
        p = PatchCollection(patches, cmap = cmap, edgecolor = 'black', linewidth = 0.5) #'Greys_r'
        p.set_array(colors)
        p.set_clim([-limit, limit])
        ax[k].set_axis_off()
        ax[k].add_collection(p)
    if colorbar:
        fig.colorbar(p, ax=ax[-1])
    if savefig:
        plt.savefig(path, dpi = 600)
    plt.show() 

In [ ]:
plot_polarityCircles(roi_ooindex, newcmp, limit=limit, colorbar = False)

In [ ]:
# FOR ONE FIELD: PLOT SC ON THE FIELD'S ROI MASK 

import matplotlib.cm as cm
cmap = cm.get_cmap('PiYG')
cmap.set_bad(color='grey')

ipl_mask = (Field.RoiMask() & first_field).fetch1('roi_mask')

SCmap_center = ipl_mask*(-1)
SCmap_ring = ipl_mask*(-1)
SCmap_surround = ipl_mask*(-1)

xlength = SCmap_center.shape[0]
zlength = SCmap_center.shape[1]

roi_list = first_field['roi'].values

for x in range(xlength):
    for z in range(zlength):  
        if SCmap_center[x,z]== 0:
            SCmap_center[x,z] = np.nan
        elif SCmap_center[x,z] in roi_list:
            roi_index = np.int(SCmap_center[x, z])
            SCmap_center[x,z] = first_field['spectral_contrast_center'].loc[first_field['roi'] == roi_index].values[0]
        else:
            SCmap_center[x,z] = np.nan
            
        if SCmap_ring[x,z]== 0:
            SCmap_ring[x,z] = np.nan
        elif SCmap_ring[x,z] in roi_list:
            roi_index = np.int(SCmap_ring[x, z])
            SCmap_ring[x,z] = first_field['spectral_contrast_ring'].loc[first_field['roi'] == roi_index].values[0]
        else:
            SCmap_ring[x,z] = np.nan
            
        if SCmap_surround[x,z]== 0:
            SCmap_surround[x,z] = np.nan
        elif SCmap_surround[x,z] in roi_list:
            roi_index = np.int(SCmap_surround[x, z])
            SCmap_surround[x,z] = first_field['spectral_contrast_surround'].loc[first_field['roi'] == roi_index].values[0]
        else:
            SCmap_surround[x,z] = np.nan
            
# masked_array_center = np.ma.array (SCmap_center, mask=np.isnan(SCmap_center))
# masked_array_ring = np.ma.array (SCmap_ring, mask=np.isnan(SCmap_ring))
# masked_array_surround = np.ma.array (SCmap_surround, mask=np.isnan(SCmap_surround))

fig, ax = plt.subplots(1,3, figsize=(15,15))

img= ax[0].imshow(SCmap_center.T, cmap=newcmp, vmin= -limit, vmax= limit, origin='lower') 
ax[1].imshow(SCmap_ring.T, cmap=newcmp, vmin= -limit, vmax= limit, origin='lower') 
ax[2].imshow(SCmap_surround.T, cmap=newcmp, vmin= -limit, vmax= limit, origin='lower') 

## Chromatic preference - Figure 2

In [ ]:
# get the x retinal position of each ROI
key = {
                'experimenter': 'Korympidou',
        }
x = pd.DataFrame.from_dict(((RelativeFieldLocation()) &

                                    key).fetch(as_dict=True))

df['Rel. field location x'] = ''
for index, row in df.iterrows():
    
    df.at[index, 'Rel. field location x'] = x['relx'][(x['date'] == row['date']) & (x['exp_num'] == row['exp_num']) & (x['field_id'] == row['field_id'])].values[0]

In [ ]:
# discard ROIs that have wrong y location assignment
df= df.loc[df['y_retinal_location'] != 'no_info']
df = df.reset_index(drop = True)

In [ ]:
location_y = df['y_retinal_location']
location_x = df['Rel. field location x']

In [ ]:
# set color map
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('PiYG', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([1, 1, 1, 1])

limit = 0.75 

vmax = limit 
vmin = -limit
white_start = 1/4* limit 
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
# add a random value in x & y for each ROI to distance them from one another in their field region
# value = 700 
# random_addition_1 = np.random.uniform(low=-value, high=value, size=len(df))
# random_addition_2 = np.random.uniform(low=-value, high=value, size=len(df))

random_addition_1 = np.load('random_array_xposition.npy')
random_addition_2 = np.load('random_array_yposition.npy')

In [ ]:
df['Rel. field location x + random'] = df['Rel. field location x'] + random_addition_1/1.7 
df['Rel. field location y + random'] = df['Rel. field location y'] + random_addition_2/1.7 

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(20,6))
sc=ax1.scatter(x=df['Rel. field location x + random'].values, 
                y= df['Rel. field location y + random'].values, 
                c= df['spectral_contrast_center'].values,  
                cmap=newcmp, vmax = limit, vmin= -limit, s=15) 

ax2.scatter(x=df['Rel. field location x + random'].values, 
                y= df['Rel. field location y + random'].values, 
                c= df['spectral_contrast_ring'].values, 
                cmap=newcmp, vmax = limit, vmin= -limit, s=15)

ax3.scatter(x=df['Rel. field location x + random'].values, 
                y= df['Rel. field location y + random'].values, 
                c= df['spectral_contrast_surround'].loc[df['spectral_contrast_surround']<-0.2].values, 
                cmap=newcmp, vmax=limit, vmin= -limit, s=15)
# plt.colorbar(sc)
lims = 3000
circle1=plt.Circle((0,0),lims, color='black', fill=False)
circle2=plt.Circle((0,0),lims, color='black', fill=False)
circle3=plt.Circle((0,0),lims, color='black', fill=False)
circle4=plt.Circle((0,0),50, color='black', fill=True)
circle5=plt.Circle((0,0),50, color='black', fill=True)
circle6=plt.Circle((0,0),50, color='black', fill=True)
ax1.set_xlim(-lims,lims)
ax1.set_ylim(-lims,lims)
ax2.set_xlim(-lims,lims)
ax2.set_ylim(-lims,lims)
ax3.set_xlim(-lims,lims)
ax3.set_ylim(-lims,lims)
ax1.axvline(x=0, ymin=-lims, ymax=lims)
ax2.axvline(x=0, ymin=-lims, ymax=lims)
ax3.axvline(x=0, ymin=-lims, ymax=lims)
ax1.axhline(y=0, xmin=-lims, xmax=lims)
ax2.axhline(y=0, xmin=-lims, xmax=lims)
ax3.axhline(y=0, xmin=-lims, xmax=lims)
ax1.add_patch(circle1)
ax2.add_patch(circle2)
ax3.add_patch(circle3)
ax1.add_patch(circle4)
ax2.add_patch(circle5)
ax3.add_patch(circle6)
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
plt.show()

In [ ]:
# Make bins
# y_new = (df['Rel. field location y'].unique())
# y_new.sort()
y_new= df['Rel. field location y']
bins = np.array([2192,1347.1,829.6,195.28,-633.5,-1058.4,-1340.9,-1829])
x=np.digitize(y_new,bins)
df['bin_id'] = x

In [ ]:
# Make bins - BCs

# BC_new = (df_bc['Rel. field location y'].unique())
# BC_new.sort()
BC_new= df_bc['Rel. field location y']
BC_bins = np.array([1857,1262.,785.18,344.,-1438.6,-1690.8,-1797.62,-1890])
a=np.digitize(BC_new,BC_bins)
df_bc['bin_id'] = a

In [ ]:
nb_bins = 7
fig, ax = plt.subplots(nb_bins,2, figsize=(5,10), sharex='all')

for i in range(0,nb_bins):
    sns.kdeplot(x='spectral_contrast_ring',
            data= df.loc[df['bin_id'] == i+1],linestyle='dashed', color= 'black', ax=ax[i,0])
    sns.kdeplot(x='spectral_contrast_center',
                data= df.loc[df['bin_id'] == i+1],linestyle= 'solid', color= 'black', ax=ax[i,0])
    sns.kdeplot(x='spectral_contrast_center',
                data= df_bc.loc[df_bc['bin_id'] == i+1],linestyle= 'solid', color= 'blue', ax=ax[i,0])
    
    sns.kdeplot(x='spectral_contrast_surround',
                data= df.loc[df['bin_id'] == i+1],linestyle='solid', color= 'black', ax=ax[i,1])
    sns.kdeplot(x='spectral_contrast_surround',
                data= df_bc.loc[df_bc['bin_id'] == i+1],linestyle='solid', color= 'blue', ax=ax[i,1])
    ax[6,0].axvline(0, color = 'black', linestyle= '--')
    ax[6,1].axvline(0, color = 'black', linestyle= '--')
    plt.xlim(-1.5,1.5)
plt.show()

### Amacrine cells

In [ ]:
# Heatmap of average SC per bin

bin_id= df['bin_id']
nb_bins = 7

mean_bin_contrast= np.zeros(((nb_bins),3))

for i in range(0,nb_bins):
    
    mean_bin_contrast[i,0]= df['spectral_contrast_center'][np.where(bin_id == i+1)[0]].mean()
    mean_bin_contrast[i,1]= df['spectral_contrast_ring'][np.where(bin_id == i+1)[0]].mean()
    mean_bin_contrast[i,2]= df['spectral_contrast_surround'][np.where(bin_id == i+1)[0]].mean()
sns.heatmap(mean_bin_contrast,cmap=newcmp,vmin=-limit,vmax=limit)

plt.xticks([0.5, 1.5, 2.5], ['Center', 'Ring', 'Surround'])
plt.show()

In [ ]:
# set color map
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('PiYG', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([1, 1, 1, 1])

limit = np.amax((mean_bin_contrast.max(), np.abs(mean_bin_contrast.min())))

vmax = limit # These values change if you work w drug data
vmin = -limit
white_start = 1/4* limit  #0.09 # Choose yourself as you like and keep stable for all further analysis
white_stop = -1/4* limit 

# To explain white borders take 1/4 of the max SC value across clusters and spatial conditions
# np.amax((np.abs(mean_cluster_spectral_contrast.min()), np.abs(mean_cluster_spectral_contrast.max())))

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range#*2
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

### Bipolar cells

In [ ]:
# Heatmap of average SC per bin - BIPOLAR CELLS

bin_id= df_bc['bin_id']
nb_bins = 7

bc_mean_bin_contrast= np.zeros(((nb_bins),2))

for i in range(0,nb_bins):
    
    bc_mean_bin_contrast[i,0]= df_bc['spectral_contrast_center'][np.where(bin_id == i+1)[0]].mean()
    bc_mean_bin_contrast[i,1]= df_bc['spectral_contrast_surround'][np.where(bin_id == i+1)[0]].mean()

sns.heatmap(bc_mean_bin_contrast,cmap=newcmp,vmin=-limit,vmax=limit)

plt.xticks([0.5, 1.5], ['Center', 'Surround'])
plt.show()

In [ ]:
# Make bins across the IPL

depth_new= df['ipl_depth']
bins = np.array([0,0.2,0.4,0.6,0.8,1.01])
# bins = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.01])
x=np.digitize(depth_new,bins)
df['depth_id'] = x

bc_depth_new= df_bc['ipl_depth']
bc_bins = np.array([0,0.2,0.4,0.6,0.8,1.01])
a=np.digitize(bc_depth_new,bc_bins)
df_bc['depth_id'] = a

In [ ]:
df_dorsal= df.loc[df['y_retinal_location'] == 'dorsal']
df_ventral= df.loc[df['y_retinal_location'] == 'ventral']

bc_df_dorsal= df_bc.loc[df_bc['Rel. field location y'] > 0]
bc_df_ventral= df_bc.loc[df_bc['Rel. field location y'] < 0]

In [ ]:
nb_bins = 5

mean_bin_contrast_d= np.zeros(((nb_bins),3))
mean_bin_contrast_v= np.zeros(((nb_bins),3))
bc_mean_bin_contrast_d= np.zeros(((nb_bins),2))
bc_mean_bin_contrast_v= np.zeros(((nb_bins),2))

for i in range(0,nb_bins):
    
    mean_bin_contrast_d[i,0]= df_dorsal['spectral_contrast_center'].loc[df_dorsal['depth_id'] == i+1].values.mean()
    mean_bin_contrast_d[i,1]= df_dorsal['spectral_contrast_ring'].loc[df_dorsal['depth_id'] == i+1].values.mean()
    mean_bin_contrast_d[i,2]= df_dorsal['spectral_contrast_surround'].loc[df_dorsal['depth_id'] == i+1].values.mean()

    bc_mean_bin_contrast_d[i,0]= bc_df_dorsal['spectral_contrast_center'].loc[bc_df_dorsal['depth_id'] == i+1].values.mean()
    bc_mean_bin_contrast_d[i,1]= bc_df_dorsal['spectral_contrast_surround'].loc[bc_df_dorsal['depth_id'] == i+1].values.mean()
    
    mean_bin_contrast_v[i,0]= df_ventral['spectral_contrast_center'].loc[df_ventral['depth_id'] == i+1].values.mean()
    mean_bin_contrast_v[i,1]= df_ventral['spectral_contrast_ring'].loc[df_ventral['depth_id'] == i+1].values.mean()
    mean_bin_contrast_v[i,2]= df_ventral['spectral_contrast_surround'].loc[df_ventral['depth_id'] == i+1].values.mean()

    bc_mean_bin_contrast_v[i,0]= bc_df_ventral['spectral_contrast_center'].loc[bc_df_ventral['depth_id'] == i+1].values.mean()
    bc_mean_bin_contrast_v[i,1]= bc_df_ventral['spectral_contrast_surround'].loc[bc_df_ventral['depth_id'] == i+1].values.mean() 

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(8,15),sharex='all',sharey='all')

sns.histplot(data=df_ventral, 
                x='spectral_contrast_center', 
                y= 'ipl_depth',ax=ax[0,0],bins=40,cmap='binary',vmax=5)
sns.histplot(data=df_ventral, 
                x='spectral_contrast_ring', 
                y= 'ipl_depth',ax=ax[1,0],bins=40,cmap='binary',vmax=5)
sns.histplot(data=df_ventral, 
                x='spectral_contrast_surround', 
                y= 'ipl_depth',ax=ax[2,0],bins=40,cmap='binary',vmax=5)
ax[0,0].plot(mean_bin_contrast_v[:,0],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'white')
ax[1,0].plot(mean_bin_contrast_v[:,1],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'white')
ax[2,0].plot(mean_bin_contrast_v[:,2],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'white')
ax[0,0].plot(bc_mean_bin_contrast_v[:,0],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'blue')
ax[2,0].plot(bc_mean_bin_contrast_v[:,1],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'blue')

sns.histplot(data=df_dorsal, 
                x='spectral_contrast_center', 
                y= 'ipl_depth',ax=ax[0,1],bins=40,cmap='binary',vmax=5)
sns.histplot(data=df_dorsal, 
                x='spectral_contrast_ring', 
                y= 'ipl_depth',ax=ax[1,1],bins=40,cmap='binary',vmax=5)
sns.histplot(data=df_dorsal, 
                x='spectral_contrast_surround', 
                y= 'ipl_depth',ax=ax[2,1],bins=40,cmap='binary',vmax=5)

ax[0,1].plot(mean_bin_contrast_d[:,0],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'white')
ax[1,1].plot(mean_bin_contrast_d[:,1],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'white')
ax[2,1].plot(mean_bin_contrast_d[:,2],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'white')
ax[0,1].plot(bc_mean_bin_contrast_d[:,0],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'blue')
ax[2,1].plot(bc_mean_bin_contrast_d[:,1],[0.0,0.25,0.5,0.75,1.0],'-',linewidth=4,color= 'blue')

ax[0,0].set_title('Ventral')
ax[0,1].set_title('Dorsal')
ax[2,0].axvline(0, color = 'red', linestyle='dashed')
ax[2,1].axvline(0, color = 'red', linestyle='dashed')

plt.xlim(-1.2,1.2)

plt.show()

### Amacrine cells

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))

sns.heatmap(mean_bin_contrast_v,cmap=newcmp,vmin=-limit,vmax=limit,ax=ax[0])
sns.heatmap(mean_bin_contrast_d,cmap=newcmp,vmin=-limit,vmax=limit,ax=ax[1])
ax[0].set_title('Ventral')
ax[1].set_title('Dorsal')
ax[0].set_xticks([0.5, 1.5, 2.5])
ax[0].set_xticklabels(['Center', 'Ring', 'Surround'])
ax[1].set_xticks([0.5, 1.5, 2.5])
ax[1].set_xticklabels(['Center', 'Ring', 'Surround'])
ax[0].invert_yaxis()
ax[1].invert_yaxis()

# plt.savefig('SC_ipldistribution_mean_per_bin.eps', format='eps')
plt.show()

### Bipolar cells

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))

sns.heatmap(bc_mean_bin_contrast_v,cmap=newcmp,vmin=-limit,vmax=limit,ax=ax[0])
sns.heatmap(bc_mean_bin_contrast_d,cmap=newcmp,vmin=-limit,vmax=limit,ax=ax[1])
ax[0].set_title('Ventral')
ax[1].set_title('Dorsal')
ax[0].set_xticks([0.5, 1.5])
ax[0].set_xticklabels(['Center', 'Surround'])
ax[1].set_xticks([0.5, 1.5])
ax[1].set_xticklabels(['Center', 'Surround'])
ax[0].invert_yaxis()
ax[1].invert_yaxis()

plt.show()

## Clustering amacrine cells - Figure 3

In [ ]:
# Plot kernel heatmap
name = 'cluster ID (diag)'
plot_Kernels(df, name)

In [ ]:
# Plot location-ipldepth histos
df_part= df.loc[df['y_retinal_location'] != 'no_info']
df_part = df_part.reset_index(drop = True)
name = 'cluster ID (diag)'
plot_histogram(df_part, name)

In [ ]:
# Plot kernel averages
def plot_Kernels_MK(dataframe, clustering_name,  nb_clusters = None, savefig = False, path_means = None):
    size_kernel = len(dataframe['uv_center'].iloc[0])
    assert size_kernel == 1250
    labels = ['UV-c', 'UV-r', 'UV-s', 'G-c', 'G-r', 'G-s']
    # colors_uv = ['purple', 'purple', 'purple', 'darkgreen', 'limegreen', 'lime']
    cluster_IDs = dataframe[clustering_name].to_numpy()
    assert np.amin(cluster_IDs) == 0
    if nb_clusters is None:
        nb_clusters = np.unique(cluster_IDs).shape[0]
    orderROIs = np.argsort(cluster_IDs)
    cluster_sizes = np.zeros(nb_clusters)
    for index in range(nb_clusters):
        cluster_sizes[index] = np.where(cluster_IDs == index)[0].shape[0]
    assert cluster_sizes.sum() == len(dataframe)
    data_uv = np.concatenate((np.vstack(dataframe['uv_center'].to_numpy()),
                              np.vstack(dataframe['uv_ring'].to_numpy()),
                              np.vstack(dataframe['uv_surround'].to_numpy())), axis = 1)
    data_green = np.concatenate((np.vstack(dataframe['green_center'].to_numpy()),
                                 np.vstack(dataframe['green_ring'].to_numpy()),
                                 np.vstack(dataframe['green_surround'].to_numpy())), axis = 1)

    uv_center_amplitude = dataframe['uv_center amplitude']
    uv_ring_amplitude = dataframe['uv_ring amplitude']
    uv_surround_amplitude = dataframe['uv_surround amplitude']
    green_center_amplitude = dataframe['green_center amplitude']
    green_ring_amplitude = dataframe['green_ring amplitude']
    green_surround_amplitude = dataframe['green_surround amplitude']
    
    cluster_means_uv = np.zeros((nb_clusters,size_kernel*3))
    cluster_means_green = np.zeros((nb_clusters,size_kernel*3))
    cluster_amplitude_uv = np.zeros((nb_clusters,3))
    cluster_amplitude_green = np.zeros((nb_clusters,3))
    std_amplitude_uv = np.zeros((nb_clusters,3))
    std_amplitude_green = np.zeros((nb_clusters,3))

    for current_cluster_ID in range(nb_clusters): 
        cluster_mask = np.where(cluster_IDs == current_cluster_ID)[0]
        
        current_data_uv = data_uv[cluster_mask,:]
        current_data_green = data_green[cluster_mask,:]
        cluster_means_uv[current_cluster_ID,:]  = np.mean(current_data_uv, axis = 0)
        cluster_means_green[current_cluster_ID,:]  = np.mean(current_data_green, axis = 0)
        
        cluster_amplitude_uv[current_cluster_ID,0] = uv_center_amplitude[cluster_mask].mean()
        cluster_amplitude_uv[current_cluster_ID,1] = uv_ring_amplitude[cluster_mask].mean()
        cluster_amplitude_uv[current_cluster_ID,2] = uv_surround_amplitude[cluster_mask].mean()
        cluster_amplitude_green[current_cluster_ID,0] = green_center_amplitude[cluster_mask].mean()
        cluster_amplitude_green[current_cluster_ID,1] = green_ring_amplitude[cluster_mask].mean()
        cluster_amplitude_green[current_cluster_ID,2] = green_surround_amplitude[cluster_mask].mean()
        
        std_amplitude_uv[current_cluster_ID,0] = np.std(uv_center_amplitude[cluster_mask])
        std_amplitude_uv[current_cluster_ID,1] = np.std(uv_ring_amplitude[cluster_mask])
        std_amplitude_uv[current_cluster_ID,2] = np.std(uv_surround_amplitude[cluster_mask])
        std_amplitude_green[current_cluster_ID,0] = np.std(green_center_amplitude[cluster_mask])
        std_amplitude_green[current_cluster_ID,1] = np.std(green_ring_amplitude[cluster_mask])
        std_amplitude_green[current_cluster_ID,2] = np.std(green_surround_amplitude[cluster_mask])
   
    fig, ax = plt.subplots(nb_clusters, 6, sharey='all', figsize=(5,20))  

    bar_colors = ['purple', 'darkgreen']

    for current_cluster_ID in range(nb_clusters):
        for current_kernel_ID in range(3):
            #plotting kernels
            start_kernel = current_kernel_ID*size_kernel
            my_ax = ax[current_cluster_ID, current_kernel_ID*2] 

            my_ax.axis('off')
            my_ax.axhline(0, color = 'black', linestyle = 'dashed', linewidth = 0.75)
            my_ax.axvline(size_kernel*3/4, color = 'black', linestyle = 'dashed', linewidth = 0.75) 
            my_ax.plot(cluster_means_uv[current_cluster_ID,start_kernel:start_kernel+size_kernel],
                       color = 'purple', linewidth = 1.0)
            my_ax.plot(cluster_means_green[current_cluster_ID,start_kernel:start_kernel+size_kernel],
                       color = 'darkgreen', linewidth = 1.0)
            if current_cluster_ID == 0:
                my_ax.set_title(labels[current_kernel_ID])
            if current_kernel_ID == 0:
                my_ax.vlines(x=[0], ymin=[0], ymax=[0.1], lw=1) #scale bar 0.1
                
            #plotting bars
            second_ax = ax[current_cluster_ID, current_kernel_ID*2+1] 
            second_ax.bar([0,1],[cluster_amplitude_uv[current_cluster_ID,current_kernel_ID], 
                                 cluster_amplitude_green[current_cluster_ID,current_kernel_ID]],
                          yerr=[std_amplitude_uv[current_cluster_ID,current_kernel_ID], 
                                 std_amplitude_green[current_cluster_ID,current_kernel_ID]],
                          align='center', capsize=2, width=1, color=bar_colors) 
            second_ax.axhline(linestyle='dotted',c='k')
            second_ax.axis('off')  

    if savefig:
        plt.savefig(path_means, dpi = 600, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot kernel averages
name = 'cluster ID (diag)'
plot_Kernels_MK(df, name)

In [ ]:
# Plot SC rings
mean_cluster_spectral_contrast = np.zeros(((nb_clusters),3))

for i in range(nb_clusters):

    mean_cluster_spectral_contrast[i,0]= df['spectral_contrast_surround'].loc[df['cluster ID (diag)'] == i].mean() 
    mean_cluster_spectral_contrast[i,1]= df['spectral_contrast_ring'].loc[df['cluster ID (diag)'] == i].mean()
    mean_cluster_spectral_contrast[i,2]= df['spectral_contrast_center'].loc[df['cluster ID (diag)'] == i].mean()

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('PiYG', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([1, 1, 1, 1])

limit = np.amax((np.amax(mean_cluster_spectral_contrast), np.abs(np.amin(mean_cluster_spectral_contrast))))

vmax = limit 
vmin = -limit

# To explain white borders take 1/4 of the max SC value across clusters and spatial conditions
white_start = 1/4* limit 
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range#*2
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

def plot_colorCircles(color_pref, cmap, limit = None, colorbar = False, savefig = False, path = None):
    center = [0.5, 0.5]
    radii = [0.3, 0.2, 0.1]
    nb_clusters = color_pref.shape[0]
    if limit is None:
        limit = np.amax((np.amax(color_pref), np.abs(np.amin(color_pref))))
    fig, ax = plt.subplots(nb_clusters, 1, figsize=(1.3,nb_clusters*1.5))
    for k in range(nb_clusters):
        patches = []
        colors = color_pref[k,:]
        for r in radii: 
            circle = Circle((center), r)
            patches.append(circle)
        p = PatchCollection(patches, cmap = cmap, edgecolor = 'black', linewidth = 0.5)
        p.set_array(colors)
        p.set_clim([-limit, limit])
        ax[k].set_axis_off()
        ax[k].add_collection(p)
    if colorbar:
        fig.colorbar(p, ax=ax[-1])
    if savefig:
        plt.savefig(path, dpi = 600)
    plt.show() 

In [ ]:
plot_colorCircles(mean_cluster_spectral_contrast, newcmp, limit=None, colorbar=True)

In [ ]:
# Plot OOi rings
mean_ooindex= np.zeros(((nb_clusters),3))

for i in range(nb_clusters):
    mean_ooindex[i,0] = ooindex[np.where(df['cluster ID (diag)'] == i), 2].mean() #mean surround
    mean_ooindex[i,1] = ooindex[np.where(df['cluster ID (diag)'] == i), 1].mean()
    mean_ooindex[i,2] = ooindex[np.where(df['cluster ID (diag)'] == i), 0].mean()

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('Greys_r', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([0.5, 0.5, 0.5, 0.5])

limit = np.amax((np.amax(mean_ooindex), np.abs(np.amin(mean_ooindex))))

vmax = limit 
vmin = -limit

white_start = 1/4* limit
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

def plot_polarityCircles(color_pref, cmap, limit = None, colorbar = False, savefig = False, path = None):
    center = [0.5, 0.5]
    radii = [0.3, 0.2, 0.1]
    nb_clusters = color_pref.shape[0]
    if limit is None:
        limit = np.amax((np.amax(color_pref), np.abs(np.amin(color_pref))))
    fig, ax = plt.subplots(nb_clusters, 1, figsize=(1.3,nb_clusters*1.5))
    for k in range(nb_clusters):
        patches = []
        colors = color_pref[k,:]
        for r in radii: 
            circle = Circle((center), r)
            patches.append(circle)
        p = PatchCollection(patches, cmap = cmap, edgecolor = 'black', linewidth = 0.5) #'Greys_r'
        p.set_array(colors)
        p.set_clim([-limit, limit])
        ax[k].set_axis_off()
        ax[k].add_collection(p)
    if colorbar:
        fig.colorbar(p, ax=ax[-1])
    if savefig:
        plt.savefig(path, dpi = 600)
    plt.show() 

In [ ]:
plot_polarityCircles(mean_ooindex, newcmp, limit=None, colorbar = True)

## Chromatic tuning organizing principles - Figure 4

In [ ]:
# use chromatic kernels and group clusters together

mean_cluster_uv_center= np.zeros(((nb_clusters),1250))
mean_cluster_uv_ring= np.zeros(((nb_clusters),1250))
mean_cluster_uv_surround= np.zeros(((nb_clusters),1250))
mean_cluster_green_center= np.zeros(((nb_clusters),1250))
mean_cluster_green_ring= np.zeros(((nb_clusters),1250))
mean_cluster_green_surround= np.zeros(((nb_clusters),1250))

for i in range(nb_clusters):
    mean_cluster_uv_center[i,:]= np.stack(df['uv_center'].loc[df['cluster ID (diag)'] == i]).mean(axis=0)
    mean_cluster_uv_ring[i,:]= np.stack(df['uv_ring'].loc[df['cluster ID (diag)'] == i]).mean(axis=0)
    mean_cluster_uv_surround[i,:]= np.stack(df['uv_surround'].loc[df['cluster ID (diag)'] == i]).mean(axis=0)
    mean_cluster_green_center[i,:]= np.stack(df['green_center'].loc[df['cluster ID (diag)'] == i]).mean(axis=0)
    mean_cluster_green_ring[i,:]= np.stack(df['green_ring'].loc[df['cluster ID (diag)'] == i]).mean(axis=0)
    mean_cluster_green_surround[i,:]= np.stack(df['green_surround'].loc[df['cluster ID (diag)'] == i]).mean(axis=0)

In [ ]:
array = np.concatenate((mean_cluster_uv_center,mean_cluster_uv_ring,mean_cluster_uv_surround,
                       mean_cluster_green_center,mean_cluster_green_ring,mean_cluster_green_surround),axis=1)

In [ ]:
array_norm = np.zeros((nb_clusters,array.shape[1]))

for i in range(nb_clusters):
    array_norm[i,:] = array[i,:]/(max(np.amax(array[i,:]),np.abs(np.amin(array[i,:]))))

In [ ]:
from sklearn.metrics import mean_squared_error

corr_dist_features = np.zeros((nb_clusters,nb_clusters))

for i in range(nb_clusters):
    for j in range(nb_clusters):
        corr_dist_features[i,j] = mean_squared_error(array_norm[i,:],array_norm[j,:])

In [ ]:
# distance dendrograms or ACs & BCs
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, linkage

dists = squareform(corr_dist_features, checks=False)

fig = plt.figure(figsize=(15,10))
linkage_matrix = linkage(dists, "weighted",optimal_ordering=True)

a=dendrogram(linkage_matrix,orientation='top')

plt.show()

## Clustering bipolar cells - Supplementary Figure 1

In [ ]:
# Plot kernel heatmap
def updated_plot_heatmap_BIPOLAR_cells(dataframe, clustering_name,  nb_clusters = None,
                                       savefig = False, path_ROIs = None, path_means = None):
    size_kernel = len(dataframe['uv_center'].iloc[0])
    assert size_kernel == 1250
    labels = ['UV-c', 'UV-s', 'G-c', 'G-s']
    cluster_IDs_original = dataframe[clustering_name].to_numpy() # Based on IPL depth
    cluster_IDs = np.zeros(len(cluster_IDs_original))
    # Manually defined based on how we want to display the clusters
    # Based on 1) dorso-ventral location and then on 2) IPL depth
    order = np.array([2,5,7,10,12,14,16,21,22,23,3,4,6,8,9,11,13,15,17,18,19,20,24,1]) - 1
    #2,5,7,10,12,14,16,21,22,23,3,6,8,9,11,15,18,19,20,1,4,13,17,24
    for new_id, old_id in enumerate(order):
        cluster_IDs[np.where(cluster_IDs_original == int(old_id))[0]] = int(new_id)
    assert np.amin(cluster_IDs) == 0
    if nb_clusters is None:
        nb_clusters = np.unique(cluster_IDs).shape[0]
    orderROIs = np.argsort(cluster_IDs)
    cluster_sizes = np.zeros(nb_clusters)
    for index in range(nb_clusters):
        cluster_sizes[index] = np.where(cluster_IDs == index)[0].shape[0]
    assert cluster_sizes.sum() == len(dataframe)
    data = np.concatenate((np.vstack(dataframe['uv_center'].to_numpy()),
                           np.vstack(dataframe['uv_surround'].to_numpy()),
                           np.vstack(dataframe['green_center'].to_numpy()),
                           np.vstack(dataframe['green_surround'].to_numpy())), axis = 1)
    plt.figure(figsize=(8,15))
    plt.imshow(data[orderROIs,:], aspect = 'auto', cmap = 'binary_r', interpolation = 'None')
    plt.xticks(np.array([(size_kernel/2)+i*size_kernel for i in range(4)]) - 0.5, labels)
    plt.yticks(np.array([cluster_sizes[i]/2 + cluster_sizes[:i].sum() for i in range(nb_clusters)]) - 0.5,
               ['C$_{' +  str(i) + "}$" for i in range(nb_clusters)])
    plt.tick_params(length = 0)
    for index in range(1, 4):
        plt.axvline((index*size_kernel)-0.5, color = 'white', linewidth = 0.75)
    current_line_location = -0.5
    for cluster_id in range(nb_clusters-1):
        current_line_location = current_line_location + cluster_sizes[cluster_id]
        plt.axhline(current_line_location, color = 'white', linewidth = 0.75)
    if savefig:
        plt.savefig(path_ROIs, dpi = 600, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot kernel heatmap
name = 'cluster ID (full)'
updated_plot_heatmap_BIPOLAR_cells(df_bc, name)

In [ ]:
# Plot location-ipldepth histos
def updated_plot_histogram_BIPOLAR_cells(dataframe, clustering_name, savefig = False, path = None):   
    cluster_IDs_original = dataframe[clustering_name].to_numpy() # Based on IPL depth
    cluster_IDs = np.zeros(len(cluster_IDs_original))
    # Manually defined based on how we want to display the clusters
    # Based on 1) dorso-ventral location and then on 2) IPL depth
    order = np.array([2,5,7,10,12,14,16,21,22,23,3,6,8,9,11,15,18,19,20,1,4,13,17,24]) - 1
    for new_id, old_id in enumerate(order):
        cluster_IDs[np.where(cluster_IDs_original == int(old_id))[0]] = int(new_id)
    
    nb_clusters = np.unique(cluster_IDs).shape[0]
    y_location = dataframe['Rel. field location y'].to_numpy()
    IPL_depth = dataframe['ipl_depth'].to_numpy()

    # Hist for V-D location
    max_y_location = np.amax(y_location)
    min_y_location = np.amin(y_location)
    total_max_y = np.amax((max_y_location, np.abs(min_y_location)))
    binwidth_y_location = 200
    bins_y = np.arange(min_y_location, max_y_location + binwidth_y_location, binwidth_y_location)

    # Hist for IPL depth
    max_IPL = np.amax(IPL_depth)
    min_IPL = np.amin(IPL_depth)
    binwidth_IPL = 0.05
    bins_IPL = np.arange(min_IPL, max_IPL + binwidth_IPL, binwidth_IPL)

    fig, ax = plt.subplots(nb_clusters, 2, sharex='col', sharey='col', figsize=(4,15))
    for current_cluster_ID in range(nb_clusters):

        cluster_mask = np.where(cluster_IDs == current_cluster_ID)[0]
        current_y = y_location[cluster_mask]
        current_IPL = IPL_depth[cluster_mask]

        # V-D 
        my_ax = ax[current_cluster_ID, 0]
        my_ax.axvline(0, color = 'black', linewidth = 0.5)
        my_ax.hist(current_y, bins = bins_y, color = 'black')
        my_ax.set_xlim([-total_max_y-300,total_max_y+300])

        if current_cluster_ID == 0:
            my_ax.set_title('V-D location')
        if current_cluster_ID == (nb_clusters - 1):
            my_ax.set_ylabel('Nb of Rois')
            my_ax.set_xlabel('Ventral - dorsal \n(Rel. y location)')
        else:
            my_ax.tick_params(axis='x', which='both', bottom=False)
            my_ax.tick_params(axis='y', which='both', left=False, labelsize=0.0, labelcolor = 'white')

        #IPL
        my_ax = ax[current_cluster_ID, 1]
        my_ax.axvline(0, color = 'black', linewidth = 0.5)
        my_ax.axvline(1, color = 'black', linewidth = 0.5)
        my_ax.hist(current_IPL, bins = bins_IPL, color = 'black')

        if current_cluster_ID == 0:
            my_ax.set_title('IPL depth')
        if current_cluster_ID == (nb_clusters - 1):
            my_ax.set_xlabel('IPL depth')
            my_ax.set_xticks([0,1])    
            my_ax.set_xticklabels(['GCL', 'INL'])
        else:
            my_ax.tick_params(axis='x', which='both', bottom=False)            
            my_ax.tick_params(axis='y', which='both', left=False, labelsize=0.0, labelcolor = 'white')

    if savefig:
        plt.savefig(path, dpi = 600, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot location-ipldepth histos
name = 'cluster ID (full)'
updated_plot_histogram_BIPOLAR_cells(df_bc, name)

In [ ]:
# Plot kernel averages
def updated_plot_Kernels_BIPOLAR_cells(dataframe, clustering_name,  nb_clusters = None, savefig = False, path_means = None):
    size_kernel = len(dataframe['uv_center'].iloc[0])
    assert size_kernel == 1250
    labels = ['UV-c', 'UV-s', 'G-c', 'G-s']
    cluster_IDs_original = dataframe[clustering_name].to_numpy() # Based on IPL depth
    cluster_IDs = np.zeros(len(cluster_IDs_original))
    # Manually defined based on how we want to display the clusters
    # Based on 1) dorso-ventral location and then on 2) IPL depth
    order = np.array([2,5,7,10,12,14,16,21,22,23,3,6,8,9,11,15,18,19,20,1,4,13,17,24]) - 1
    for new_id, old_id in enumerate(order):
        cluster_IDs[np.where(cluster_IDs_original == int(old_id))[0]] = int(new_id)
    assert np.amin(cluster_IDs) == 0
    if nb_clusters is None:
        nb_clusters = np.unique(cluster_IDs).shape[0]
    orderROIs = np.argsort(cluster_IDs)
    cluster_sizes = np.zeros(nb_clusters)
    for index in range(nb_clusters):
        cluster_sizes[index] = np.where(cluster_IDs == index)[0].shape[0]
    assert cluster_sizes.sum() == len(dataframe)
    data_uv = np.concatenate((np.vstack(dataframe['uv_center'].to_numpy()),
                              np.vstack(dataframe['uv_surround'].to_numpy())), axis = 1)
    data_green = np.concatenate((np.vstack(dataframe['green_center'].to_numpy()),
                                 np.vstack(dataframe['green_surround'].to_numpy())), axis = 1)
    uv_center_amplitude = dataframe['uv_center amplitude']
    uv_surround_amplitude = dataframe['uv_surround amplitude']
    green_center_amplitude = dataframe['green_center amplitude']
    green_surround_amplitude = dataframe['green_surround amplitude']
    cluster_means_uv = np.zeros((nb_clusters,size_kernel*2))
    cluster_means_green = np.zeros((nb_clusters,size_kernel*2))
    cluster_amplitude_uv = np.zeros((nb_clusters,2))
    cluster_amplitude_green = np.zeros((nb_clusters,2))
    std_amplitude_uv = np.zeros((nb_clusters,2))
    std_amplitude_green = np.zeros((nb_clusters,2))
    for current_cluster_ID in range(nb_clusters): # Calculate cluster means
        cluster_mask = np.where(cluster_IDs == current_cluster_ID)[0]
        current_data_uv = data_uv[cluster_mask,:]
        current_data_green = data_green[cluster_mask,:]
        cluster_means_uv[current_cluster_ID,:]  = np.mean(current_data_uv, axis = 0)
        cluster_means_green[current_cluster_ID,:]  = np.mean(current_data_green, axis = 0)
        cluster_amplitude_uv[current_cluster_ID,0] = uv_center_amplitude[cluster_mask].mean()
        cluster_amplitude_uv[current_cluster_ID,1] = uv_surround_amplitude[cluster_mask].mean()
        cluster_amplitude_green[current_cluster_ID,0] = green_center_amplitude[cluster_mask].mean()
        cluster_amplitude_green[current_cluster_ID,1] = green_surround_amplitude[cluster_mask].mean()
        std_amplitude_uv[current_cluster_ID,0] = np.std(uv_center_amplitude[cluster_mask])
        std_amplitude_uv[current_cluster_ID,1] = np.std(uv_surround_amplitude[cluster_mask])
        std_amplitude_green[current_cluster_ID,1] = np.std(green_center_amplitude[cluster_mask])
        std_amplitude_green[current_cluster_ID,1] = np.std(green_surround_amplitude[cluster_mask])
#     fig, ax = plt.subplots(nb_clusters, 2, sharex='all', sharey='row', figsize=(5,20))
    fig, ax = plt.subplots(nb_clusters, 4, sharey='row', figsize=(5,20))
    bar_colors = ['purple', 'darkgreen']
    for current_cluster_ID in range(nb_clusters):
        for current_kernel_ID in range(2):
            #plotting kernels
            start_kernel = current_kernel_ID*size_kernel
#             my_ax = ax[current_cluster_ID, current_kernel_ID]
            my_ax = ax[current_cluster_ID, current_kernel_ID*2]
            my_ax.axis('off')
            my_ax.axhline(0, color = 'black', linestyle = 'dashed', linewidth = 0.75)
            my_ax.axvline(size_kernel*3/4, color = 'black', linestyle = 'dashed', linewidth = 0.75)
            my_ax.plot(cluster_means_uv[current_cluster_ID,start_kernel:start_kernel+size_kernel],
                       color = 'purple', linewidth = 1.0)
            my_ax.plot(cluster_means_green[current_cluster_ID,start_kernel:start_kernel+size_kernel],
                       color = 'darkgreen', linewidth = 1.0)
            if current_cluster_ID == 0:
                my_ax.set_title(labels[current_kernel_ID])
            if current_kernel_ID == 0:
                my_ax.vlines(x=[0], ymin=[0], ymax=[0.2], lw=1) #scale bar 0.2
            #plotting bars
            second_ax = ax[current_cluster_ID, current_kernel_ID*2+1]
            second_ax.bar([0,1],[cluster_amplitude_uv[current_cluster_ID,current_kernel_ID],
                                 cluster_amplitude_green[current_cluster_ID,current_kernel_ID]],
                          yerr=[std_amplitude_uv[current_cluster_ID,current_kernel_ID],
                                 std_amplitude_green[current_cluster_ID,current_kernel_ID]],
                          align='center', capsize=2, width=1, color=bar_colors)
            second_ax.axhline(linestyle='dotted',c='k')
            second_ax.axis('off')
    if savefig:
        plt.savefig(path_means, dpi = 600, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot kernel averages
name = 'cluster ID (full)'
updated_plot_Kernels_BIPOLAR_cells(df_bc, name)

In [ ]:
# Plot SC rings
bc_mean_cluster_spectral_contrast = np.zeros(((bc_nb_clusters),2))
for i in range(bc_nb_clusters):
    bc_mean_cluster_spectral_contrast[i,0]= df_bc['spectral_contrast_surround'][np.where(bc_cluster_id == i)[0]].mean() # surround first for plot with circles
    bc_mean_cluster_spectral_contrast[i,1]= df_bc['spectral_contrast_center'][np.where(bc_cluster_id == i)[0]].mean()

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('PiYG', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([1, 1, 1, 1])

limit = np.amax((np.amax(mean_cluster_spectral_contrast), np.abs(np.amin(mean_cluster_spectral_contrast))))

vmax = limit 
vmin = -limit

white_start = 1/4* limit 
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

def plot_colorCircles_BIPOLAR_cells(color_pref,  cmap, limit = None, colorbar = False, savefig = False, path = None):
    center = [0.5, 0.5]
    radii = [0.3, 0.1]
    bc_nb_clusters = color_pref.shape[0]
    if limit is None:
        limit = np.amax((np.amax(color_pref), np.abs(np.amin(color_pref))))
    fig, ax = plt.subplots(bc_nb_clusters, 1, figsize=(1.3,bc_nb_clusters*1.5))
    for k in range(bc_nb_clusters):
        patches = []
        colors = color_pref[k,:]
        for r in radii: 
            circle = Circle((center), r)
            patches.append(circle)
        p = PatchCollection(patches, cmap = cmap, edgecolor = 'black', linewidth = 0.5)
        p.set_array(colors)
        p.set_clim([-limit, limit])
        ax[k].set_axis_off()
        ax[k].add_collection(p)
    if colorbar:
        fig.colorbar(p, ax=ax[-1])
    if savefig:
        plt.savefig(path, dpi = 600)
    plt.show() 

In [ ]:
plot_colorCircles_BIPOLAR_cells(bc_mean_cluster_spectral_contrast, newcmp, limit=limit, colorbar=False)

In [ ]:
# Plot OOi rings
bc_mean_ooindex= np.zeros(((bc_nb_clusters),3))

for i in range(bc_nb_clusters):
    bc_mean_ooindex[i,0] = bc_ooindex[np.where(df_bc['cluster ID (full)'] == i), 1].mean() #mean surround
    bc_mean_ooindex[i,1] = bc_ooindex[np.where(df_bc['cluster ID (full)'] == i), 0].mean()

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
PiYG = cm.get_cmap('Greys_r', 256)
new_colors = PiYG(np.linspace(0, 1, 256))
white = np.array([0.5, 0.5, 0.5, 0.5])

limit = np.amax((np.amax(mean_ooindex), np.abs(np.amin(mean_ooindex))))

vmax = limit 
vmin = -limit

white_start = 1/4* limit further analysis
white_stop = -1/4* limit 

total_range = vmax-vmin
len_color = new_colors.shape[0]
white_stop_percentage = np.abs(vmin-white_stop)*100/total_range
white_stop_pixel = int(white_stop_percentage*len_color*0.01)
white_start_percentage = np.abs(vmax-white_start)*100/total_range
white_start_pixel = int(white_start_percentage*len_color*0.01)

new_colors[white_stop_pixel:-white_start_pixel,:] = white
newcmp = ListedColormap(new_colors)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

def BIPOLAR_plot_polarityCircles(color_pref, cmap, limit = None, colorbar = False, savefig = False, path = None):
    center = [0.5, 0.5]
    radii = [0.3, 0.1]
    nb_clusters = color_pref.shape[0]
    if limit is None:
        limit = np.amax((np.amax(color_pref), np.abs(np.amin(color_pref))))
    fig, ax = plt.subplots(nb_clusters, 1, figsize=(1.3,nb_clusters*1.5))
    for k in range(nb_clusters):
        patches = []
        colors = color_pref[k,:]
        for r in radii: 
            circle = Circle((center), r)
            patches.append(circle)
        p = PatchCollection(patches, cmap = cmap, edgecolor = 'black', linewidth = 0.5) #'Greys_r'
        p.set_array(colors)
        p.set_clim([-limit, limit])
        ax[k].set_axis_off()
        ax[k].add_collection(p)
    if colorbar:
        fig.colorbar(p, ax=ax[-1])
    if savefig:
        plt.savefig(path, dpi = 600)
    plt.show() 

In [ ]:
BIPOLAR_plot_polarityCircles(bc_mean_ooindex, newcmp, limit=limit, colorbar = False)

## Chirp plotting - Supplementary Figure 2

In [ ]:
# Plot chirp averages
def plot_Chirp(dataframe, clustering_name, nb_clusters = None, savefig = False, path_ROIs = None, path_means = None):
    size_chirp = len(dataframe['global_chirp_snippets'].iloc[0])
    assert size_chirp == len(dataframe['local_chirp_snippets'].iloc[0])
    labels = ['global_chirp', 'local_chirp']
    colors = ['black', 'black'] #'#fdbb84', '#e34a33'
    cluster_IDs = dataframe[clustering_name].to_numpy()
    assert np.amin(cluster_IDs) == 0
    if nb_clusters is None:
        nb_clusters = np.unique(cluster_IDs).shape[0]
    orderROIs = np.argsort(cluster_IDs)
    cluster_sizes = np.zeros(nb_clusters)
    for index in range(nb_clusters):
        cluster_sizes[index] = np.where(cluster_IDs == index)[0].shape[0]    
    assert cluster_sizes.sum() == len(dataframe)
    data = np.concatenate((np.vstack(dataframe['global_chirp_snippets'].to_numpy()),
                           np.vstack(dataframe['local_chirp_snippets'].to_numpy())), axis = 1)
    
#     plt.figure(figsize=(8,8))
#     plt.imshow(data[orderROIs,:], aspect = 'auto', cmap = 'binary_r', interpolation = 'None', vmin=-2,vmax=4)
#     plt.xticks(np.array([(size_chirp/2)+i*size_chirp for i in range(2)]) - 0.5, labels)
#     plt.yticks(np.array([cluster_sizes[i]/2 + cluster_sizes[:i].sum() for i in range(nb_clusters)]) - 0.5,
#                ['C$_{' +  str(i) + "}$" for i in range(nb_clusters)])
#     plt.tick_params(length = 0)
#     plt.axvline(size_chirp-0.5, color = 'white', linewidth = 0.75)
#     current_line_location = -0.5
#     for cluster_id in range(nb_clusters-1):
#         current_line_location = current_line_location + cluster_sizes[cluster_id]
#         plt.axhline(current_line_location, color = 'white', linewidth = 0.75)
#     if savefig:
#         plt.savefig(path_ROIs, dpi = 600, transparent=True, bbox_inches='tight')
#     plt.show()
#     print(data.min(), data.max())
    
    cluster_means = np.zeros((nb_clusters,size_chirp*2))
    cluster_stds = np.zeros((nb_clusters,size_chirp*2))
    for current_cluster_ID in range(nb_clusters): # Calculate cluster means
        cluster_mask = np.where(cluster_IDs == current_cluster_ID)[0]
        current_data = data[cluster_mask,:]
        cluster_means[current_cluster_ID,:] = np.mean(current_data, axis = 0)
        cluster_stds[current_cluster_ID,:]  = np.std(current_data, axis = 0)
        
    fig, ax = plt.subplots(nb_clusters, 2, sharey='all',figsize=(8,25)) #sharey='all'
    for current_cluster_ID in range(nb_clusters):
        for current_kernel_ID in range(2):
            start_kernel = current_kernel_ID*size_chirp
            my_ax = ax[current_cluster_ID, current_kernel_ID]
            my_ax.axis('off')
            my_ax.axhline(0, color = 'black', linestyle = 'dashed', linewidth = 0.75)
            my_ax.plot(cluster_means[current_cluster_ID,start_kernel:start_kernel+size_chirp],
                       color = colors[current_kernel_ID], linewidth = 1.5)
            my_ax.fill_between(np.arange(size_chirp),cluster_means[current_cluster_ID,start_kernel:start_kernel+size_chirp]
                               -cluster_stds[current_cluster_ID,start_kernel:start_kernel+size_chirp],
                               cluster_means[current_cluster_ID,start_kernel:start_kernel+size_chirp]
                               +cluster_stds[current_cluster_ID,start_kernel:start_kernel+size_chirp],
                               alpha=0.1, color=colors[current_kernel_ID],zorder=5)  
            if current_cluster_ID == 0:
                my_ax.set_title(labels[current_kernel_ID])
            if current_kernel_ID == 0:
                my_ax.vlines(x=[-1], ymin=[0], ymax=[1], lw=1) #scale bar 0.3
    if savefig:
        plt.savefig(path_means, dpi = 600, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot chirp averages
df_chirp = pd.read_pickle('AC_data_unnorm.pkl')  # to include the correct chirp figures
name = 'cluster ID (diag)'
plot_Chirp(df_chirp, name)

## Pharmacology - Figure 5, Supplementary Figures 3 & 4

In [ ]:
df = pd.read_pickle('control_drug_data.pkl')
df = df.loc[(df['uv_center quality'] == 1) | (df['green_center quality'] == 1) | (df['uv_center_drug quality'] == 1) | (df['green_center_drug quality'] == 1)]
df= df.loc[df['y_retinal_location'] != 'no_info']

df = df.loc[df['treatment']=='TPMPA;gabazine']
# df = df.loc[df['treatment']=='strychnine']
# df = df.loc[df['treatment']=='LAP4']
# df = df.loc[df['treatment']=='UBP310']

df = df.reset_index(drop = True)

In [ ]:
## Calculate ROIs' SC

spectral_contrast = np.zeros((df['roi'].shape[0], 3))
drug_spectral_contrast = np.zeros((df['roi'].shape[0], 3))

for i in range(df['roi'].shape[0]):
    spectral_contrast[i,0] = (abs(df['green_center amplitude'][i]) - abs(df['uv_center amplitude'][i])) / (abs(df['green_center amplitude'][i]) + abs(df['uv_center amplitude'][i]))
    spectral_contrast[i,1] = (abs(df['green_ring amplitude'][i]) - abs(df['uv_ring amplitude'][i])) / (abs(df['green_ring amplitude'][i]) + abs(df['uv_ring amplitude'][i])) 
    spectral_contrast[i,2] = (abs(df['green_surround amplitude'][i]) - abs(df['uv_surround amplitude'][i])) / (abs(df['green_surround amplitude'][i]) + abs(df['uv_surround amplitude'][i])) 
    
    drug_spectral_contrast[i,0] = (abs(df['green_center_drug amplitude'][i]) - abs(df['uv_center_drug amplitude'][i])) / (abs(df['green_center_drug amplitude'][i]) + abs(df['uv_center_drug amplitude'][i]))
    drug_spectral_contrast[i,1] = (abs(df['green_ring_drug amplitude'][i]) - abs(df['uv_ring_drug amplitude'][i])) / (abs(df['green_ring_drug amplitude'][i]) + abs(df['uv_ring_drug amplitude'][i])) 
    drug_spectral_contrast[i,2] = (abs(df['green_surround_drug amplitude'][i]) - abs(df['uv_surround_drug amplitude'][i])) / (abs(df['green_surround_drug amplitude'][i]) + abs(df['uv_surround_drug amplitude'][i])) 

df['SC_center_ctrl'] = spectral_contrast[:,0]
df['SC_ring_ctrl'] = spectral_contrast[:,1]
df['SC_surround_ctrl'] = spectral_contrast[:,2]
df['SC_center_drug'] = drug_spectral_contrast[:,0]
df['SC_ring_drug'] = drug_spectral_contrast[:,1]
df['SC_surround_drug'] = drug_spectral_contrast[:,2]

In [ ]:
## Calculate ROIs' OOi

line_duration = 1.6 #in ms
baseline_time_ms = int(1000/line_duration)
on_time_ms = int(1000/line_duration)
off_time_ms = int(1000/line_duration)

a=np.zeros(baseline_time_ms)
b=np.ones(on_time_ms)
c=np.zeros(off_time_ms)

stimulus= np.concatenate((a,b,c))

In [ ]:
convolved_response_c = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
convolved_response_r = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
convolved_response_s = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
drug_convolved_response_c = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
drug_convolved_response_r = np.zeros((df['roi'].shape[0],stimulus.shape[0]))
drug_convolved_response_s = np.zeros((df['roi'].shape[0],stimulus.shape[0]))

line_duration_s = 0.0016
kernel_length_line = np.int(np.floor(2/line_duration_s))
offset_after = np.int(np.floor(kernel_length_line*.25)) #lines to include into the future (using 1/4 of kernel length)
offset_before = kernel_length_line-offset_after
kernel_past = offset_before - 1
kernel_future = offset_after

for i in range(df['roi'].shape[0]):
    convolved_response_uv_c = np.convolve(stimulus, np.flip(df['uv_center'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_green_c = np.convolve(stimulus, np.flip(df['green_center'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_uv_r = np.convolve(stimulus, np.flip(df['uv_ring'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_green_r = np.convolve(stimulus, np.flip(df['green_ring'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_uv_s = np.convolve(stimulus, np.flip(df['uv_surround'][i]), mode='full')[kernel_future:-kernel_past]
    convolved_response_green_s = np.convolve(stimulus, np.flip(df['green_surround'][i]), mode='full')[kernel_future:-kernel_past]
    
    convolved_response_c_avg = (convolved_response_uv_c + convolved_response_green_c)/2
    convolved_response_r_avg = (convolved_response_uv_r + convolved_response_green_r)/2
    convolved_response_s_avg = (convolved_response_uv_s + convolved_response_green_s)/2
    
    convolved_response_c[i,:] = convolved_response_c_avg-convolved_response_c_avg.min()
    convolved_response_r[i,:] = convolved_response_r_avg-convolved_response_r_avg.min()
    convolved_response_s[i,:] = convolved_response_s_avg-convolved_response_s_avg.min()
    
    drug_convolved_response_uv_c = np.convolve(stimulus, np.flip(df['uv_center_drug'][i]), mode='full')[kernel_future:-kernel_past]
    drug_convolved_response_green_c = np.convolve(stimulus, np.flip(df['green_center_drug'][i]), mode='full')[kernel_future:-kernel_past]
    drug_convolved_response_uv_r = np.convolve(stimulus, np.flip(df['uv_ring_drug'][i]), mode='full')[kernel_future:-kernel_past]
    drug_convolved_response_green_r = np.convolve(stimulus, np.flip(df['green_ring_drug'][i]), mode='full')[kernel_future:-kernel_past]
    drug_convolved_response_uv_s = np.convolve(stimulus, np.flip(df['uv_surround_drug'][i]), mode='full')[kernel_future:-kernel_past]
    drug_convolved_response_green_s = np.convolve(stimulus, np.flip(df['green_surround_drug'][i]), mode='full')[kernel_future:-kernel_past]
    
    drug_convolved_response_c_avg = (drug_convolved_response_uv_c + drug_convolved_response_green_c)/2
    drug_convolved_response_r_avg = (drug_convolved_response_uv_r + drug_convolved_response_green_r)/2
    drug_convolved_response_s_avg = (drug_convolved_response_uv_s + drug_convolved_response_green_s)/2
    
    drug_convolved_response_c[i,:] = drug_convolved_response_c_avg-drug_convolved_response_c_avg.min()
    drug_convolved_response_r[i,:] = drug_convolved_response_r_avg-drug_convolved_response_r_avg.min()
    drug_convolved_response_s[i,:] = drug_convolved_response_s_avg-drug_convolved_response_s_avg.min()    

In [ ]:
window_on_start = 625
window_on_end = 1250

window_off_start = 1250
window_off_end = 1875

ooindex = np.zeros((df['roi'].shape[0],3))
drug_ooindex = np.zeros((df['roi'].shape[0],3))

for i in range(df['roi'].shape[0]):
    ooindex[i,0] = (convolved_response_c[i,window_on_start:window_on_end].mean() - convolved_response_c[i,window_off_start:window_off_end].mean())/(convolved_response_c[i,window_on_start:window_on_end].mean() + convolved_response_c[i,window_off_start:window_off_end].mean())
    ooindex[i,1] = (convolved_response_r[i,window_on_start:window_on_end].mean() - convolved_response_r[i,window_off_start:window_off_end].mean())/(convolved_response_r[i,window_on_start:window_on_end].mean() + convolved_response_r[i,window_off_start:window_off_end].mean())
    ooindex[i,2] = (convolved_response_s[i,window_on_start:window_on_end].mean() - convolved_response_s[i,window_off_start:window_off_end].mean())/(convolved_response_s[i,window_on_start:window_on_end].mean() + convolved_response_s[i,window_off_start:window_off_end].mean())
    
    drug_ooindex[i,0] = (drug_convolved_response_c[i,window_on_start:window_on_end].mean() - drug_convolved_response_c[i,window_off_start:window_off_end].mean())/(drug_convolved_response_c[i,window_on_start:window_on_end].mean() + drug_convolved_response_c[i,window_off_start:window_off_end].mean())
    drug_ooindex[i,1] = (drug_convolved_response_r[i,window_on_start:window_on_end].mean() - drug_convolved_response_r[i,window_off_start:window_off_end].mean())/(drug_convolved_response_r[i,window_on_start:window_on_end].mean() + drug_convolved_response_r[i,window_off_start:window_off_end].mean())
    drug_ooindex[i,2] = (drug_convolved_response_s[i,window_on_start:window_on_end].mean() - drug_convolved_response_s[i,window_off_start:window_off_end].mean())/(drug_convolved_response_s[i,window_on_start:window_on_end].mean() + drug_convolved_response_s[i,window_off_start:window_off_end].mean())

df['OOi_center_ctrl'] = ooindex[:,0]
df['OOi_ring_ctrl'] = ooindex[:,1]
df['OOi_surround_ctrl'] = ooindex[:,2]
df['OOi_center_drug'] = drug_ooindex[:,0]
df['OOi_ring_drug'] = drug_ooindex[:,1]
df['OOi_surround_drug'] = drug_ooindex[:,2]

In [ ]:
# Calclate Delta SC
df['Delta SC Center'] = df['SC_center_drug'] - df['SC_center_ctrl']
df['Delta SC Ring'] = df['SC_ring_drug'] - df['SC_ring_ctrl']
df['Delta SC Surround'] = df['SC_surround_drug'] - df['SC_surround_ctrl']

# Calclate Delta OOi
df['Delta OOi Center'] = df['OOi_center_drug'] - df['OOi_center_ctrl']
df['Delta OOi Ring'] = df['OOi_ring_drug'] - df['OOi_ring_ctrl']
df['Delta OOi Surround'] = df['OOi_surround_drug'] - df['OOi_surround_ctrl']

In [ ]:
# On/Off
df_on = df.loc[(df['OOi_center_ctrl'] > 0)]
df_on = df_on.reset_index(drop = True)
df_off = df.loc[df['OOi_center_ctrl'] < 0]
df_off = df_off.reset_index(drop = True)

In [ ]:
# kdeplot On/Off 
fig, ax = plt.subplots(3,2, figsize=(10,7), sharex='all')

#center
sns.kdeplot(x='Delta OOi Center',
        data= df_on, color='darkred', ax=ax[0,0])
sns.kdeplot(x='Delta OOi Center',
        data= df_off, color='darkblue', ax=ax[0,0])
sns.kdeplot(x='Delta SC Center',
        data= df_on, color='darkred', ax=ax[0,1],label='On')
sns.kdeplot(x='Delta SC Center',
        data= df_off, color='darkblue', ax=ax[0,1],label='Off')

#ring
sns.kdeplot(x='Delta OOi Ring',
        data= df_on, color='darkred', ax=ax[1,0])
sns.kdeplot(x='Delta OOi Ring',
        data= df_off, color='darkblue', ax=ax[1,0])
sns.kdeplot(x='Delta SC Ring',
        data= df_on, color='darkred', ax=ax[1,1])
sns.kdeplot(x='Delta SC Ring',
        data= df_off, color='darkblue', ax=ax[1,1])

#surround
sns.kdeplot(x='Delta OOi Surround',
        data= df_on, color='darkred', ax=ax[2,0])
sns.kdeplot(x='Delta OOi Surround',
        data= df_off, color='darkblue', ax=ax[2,0])
sns.kdeplot(x='Delta SC Surround',
        data= df_on, color='darkred', ax=ax[2,1])
sns.kdeplot(x='Delta SC Surround',
        data= df_off, color='darkblue', ax=ax[2,1])

for current_row in range(3):
    for current_column in range(2):
        my_ax = ax[current_row,current_column] 
        my_ax.axvline(0, color='gray', linestyle='dashed')
        my_ax.set_xlim(-2,2)
        my_ax.get_yaxis().set_visible(False)
        
ax[0,0].set_title('On Off Index')
ax[0,1].set_title('Spectral Cotrast')
ax[2,0].set_xlabel('ΔSC')
ax[2,1].set_xlabel('ΔOOi')
ax[0,0].text(-2.2,0.8,'Center', size=12,verticalalignment='center',rotation='90')
ax[1,0].text(-2.2,1.2,'Ring', size=12,verticalalignment='center',rotation='90')
ax[2,0].text(-2.2,0.4,'Surround', size=12,verticalalignment='center',rotation='90')

ax[0,1].legend()
sns.despine()
plt.show()

In [ ]:
# Dorsal/Ventral
df_dorsal= df.loc[df['y_retinal_location'] == 'dorsal']
df_dorsal = df_dorsal.reset_index(drop = True)

df_ventral= df.loc[df['y_retinal_location'] == 'ventral']
df_ventral = df_ventral.reset_index(drop = True)

# On/Off & Dorsal/Ventral

df_dorsal_on = df_dorsal.loc[df_dorsal['OOi_center_ctrl'] > 0]
df_dorsal_on = df_dorsal_on.reset_index(drop = True)
df_dorsal_off = df_dorsal.loc[df_dorsal['OOi_center_ctrl'] < 0]
df_dorsal_off = df_dorsal_off.reset_index(drop = True)

df_ventral_on = df_ventral.loc[df_ventral['OOi_center_ctrl'] > 0]
df_ventral_on = df_ventral_on.reset_index(drop = True)
df_ventral_off = df_ventral.loc[df_ventral['OOi_center_ctrl'] < 0]
df_ventral_off = df_ventral_off.reset_index(drop = True)

In [ ]:
# lineplots all
fig, ax = plt.subplots(6,4, figsize=(10,15), sharex='all',sharey='all')
pos=[1,2]
label=['Ctrl','Drug']

#OOi_dorsal_on
for i in range(df_dorsal_on.shape[0]):
    ax[0,0].scatter(pos,[df_dorsal_on['OOi_center_ctrl'].values[i],df_dorsal_on['OOi_center_drug'].values[i]], color='darkred',alpha=0.05)
    ax[0,0].plot(pos,[df_dorsal_on['OOi_center_ctrl'].values[i],df_dorsal_on['OOi_center_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[2,0].scatter(pos,[df_dorsal_on['OOi_ring_ctrl'].values[i],df_dorsal_on['OOi_ring_drug'].values[i]], color='darkred',alpha=0.05)
    ax[2,0].plot(pos,[df_dorsal_on['OOi_ring_ctrl'].values[i],df_dorsal_on['OOi_ring_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[4,0].scatter(pos,[df_dorsal_on['OOi_surround_ctrl'].values[i],df_dorsal_on['OOi_surround_drug'].values[i]], color='darkred',alpha=0.05)
    ax[4,0].plot(pos,[df_dorsal_on['OOi_surround_ctrl'].values[i],df_dorsal_on['OOi_surround_drug'].values[i]], '-', color='darkred',alpha=0.05)
ax[0,0].scatter(pos,[np.mean(df_dorsal_on['OOi_center_ctrl'].values),np.mean(df_dorsal_on['OOi_center_drug'].values)], color='k')
ax[0,0].plot(pos,[np.mean(df_dorsal_on['OOi_center_ctrl'].values),np.mean(df_dorsal_on['OOi_center_drug'].values)], '-', color='k')
ax[2,0].scatter(pos,[np.mean(df_dorsal_on['OOi_ring_ctrl'].values),np.mean(df_dorsal_on['OOi_ring_drug'].values)], color='k')
ax[2,0].plot(pos,[np.mean(df_dorsal_on['OOi_ring_ctrl'].values),np.mean(df_dorsal_on['OOi_ring_drug'].values)], '-', color='k')
ax[4,0].scatter(pos,[np.mean(df_dorsal_on['OOi_surround_ctrl'].values),np.mean(df_dorsal_on['OOi_surround_drug'].values)], color='k')
ax[4,0].plot(pos,[np.mean(df_dorsal_on['OOi_surround_ctrl'].values),np.mean(df_dorsal_on['OOi_surround_drug'].values)], '-', color='k')
#OOi_ventral_on
for i in range(df_ventral_on.shape[0]):
    ax[1,0].scatter(pos,[df_ventral_on['OOi_center_ctrl'].values[i],df_ventral_on['OOi_center_drug'].values[i]], color='darkred',alpha=0.05)
    ax[1,0].plot(pos,[df_ventral_on['OOi_center_ctrl'].values[i],df_ventral_on['OOi_center_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[3,0].scatter(pos,[df_ventral_on['OOi_ring_ctrl'].values[i],df_ventral_on['OOi_ring_drug'].values[i]], color='darkred',alpha=0.05)
    ax[3,0].plot(pos,[df_ventral_on['OOi_ring_ctrl'].values[i],df_ventral_on['OOi_ring_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[5,0].scatter(pos,[df_ventral_on['OOi_surround_ctrl'].values[i],df_ventral_on['OOi_surround_drug'].values[i]], color='darkred',alpha=0.05)
    ax[5,0].plot(pos,[df_ventral_on['OOi_surround_ctrl'].values[i],df_ventral_on['OOi_surround_drug'].values[i]], '-', color='darkred',alpha=0.05)
ax[1,0].scatter(pos,[np.mean(df_ventral_on['OOi_center_ctrl'].values),np.mean(df_ventral_on['OOi_center_drug'].values)], color='k')
ax[1,0].plot(pos,[np.mean(df_ventral_on['OOi_center_ctrl'].values),np.mean(df_ventral_on['OOi_center_drug'].values)], '-', color='k')
ax[3,0].scatter(pos,[np.mean(df_ventral_on['OOi_ring_ctrl'].values),np.mean(df_ventral_on['OOi_ring_drug'].values)], color='k')
ax[3,0].plot(pos,[np.mean(df_ventral_on['OOi_ring_ctrl'].values),np.mean(df_ventral_on['OOi_ring_drug'].values)], '-', color='k')
ax[5,0].scatter(pos,[np.mean(df_ventral_on['OOi_surround_ctrl'].values),np.mean(df_ventral_on['OOi_surround_drug'].values)], color='k')
ax[5,0].plot(pos,[np.mean(df_ventral_on['OOi_surround_ctrl'].values),np.mean(df_ventral_on['OOi_surround_drug'].values)], '-', color='k')
# OOi_dorsal_off
for i in range(df_dorsal_off.shape[0]):
    ax[0,1].scatter(pos,[df_dorsal_off['OOi_center_ctrl'].values[i],df_dorsal_off['OOi_center_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[0,1].plot(pos,[df_dorsal_off['OOi_center_ctrl'].values[i],df_dorsal_off['OOi_center_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[2,1].scatter(pos,[df_dorsal_off['OOi_ring_ctrl'].values[i],df_dorsal_off['OOi_ring_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[2,1].plot(pos,[df_dorsal_off['OOi_ring_ctrl'].values[i],df_dorsal_off['OOi_ring_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[4,1].scatter(pos,[df_dorsal_off['OOi_surround_ctrl'].values[i],df_dorsal_off['OOi_surround_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[4,1].plot(pos,[df_dorsal_off['OOi_surround_ctrl'].values[i],df_dorsal_off['OOi_surround_drug'].values[i]], '-', color='darkblue',alpha=0.05)
ax[0,1].scatter(pos,[np.mean(df_dorsal_off['OOi_center_ctrl'].values),np.mean(df_dorsal_off['OOi_center_drug'].values)], color='k')
ax[0,1].plot(pos,[np.mean(df_dorsal_off['OOi_center_ctrl'].values),np.mean(df_dorsal_off['OOi_center_drug'].values)], '-', color='k')
ax[2,1].scatter(pos,[np.mean(df_dorsal_off['OOi_ring_ctrl'].values),np.mean(df_dorsal_off['OOi_ring_drug'].values)], color='k')
ax[2,1].plot(pos,[np.mean(df_dorsal_off['OOi_ring_ctrl'].values),np.mean(df_dorsal_off['OOi_ring_drug'].values)], '-', color='k')
ax[4,1].scatter(pos,[np.mean(df_dorsal_off['OOi_surround_ctrl'].values),np.mean(df_dorsal_off['OOi_surround_drug'].values)], color='k')
ax[4,1].plot(pos,[np.mean(df_dorsal_off['OOi_surround_ctrl'].values),np.mean(df_dorsal_off['OOi_surround_drug'].values)], '-', color='k')
# OOi_ventral_off
for i in range(df_ventral_off.shape[0]):
    ax[1,1].scatter(pos,[df_ventral_off['OOi_center_ctrl'].values[i],df_ventral_off['OOi_center_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[1,1].plot(pos,[df_ventral_off['OOi_center_ctrl'].values[i],df_ventral_off['OOi_center_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[3,1].scatter(pos,[df_ventral_off['OOi_ring_ctrl'].values[i],df_ventral_off['OOi_ring_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[3,1].plot(pos,[df_ventral_off['OOi_ring_ctrl'].values[i],df_ventral_off['OOi_ring_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[5,1].scatter(pos,[df_ventral_off['OOi_surround_ctrl'].values[i],df_ventral_off['OOi_surround_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[5,1].plot(pos,[df_ventral_off['OOi_surround_ctrl'].values[i],df_ventral_off['OOi_surround_drug'].values[i]], '-', color='darkblue',alpha=0.05)
ax[1,1].scatter(pos,[np.mean(df_ventral_off['OOi_center_ctrl'].values),np.mean(df_ventral_off['OOi_center_drug'].values)], color='k')
ax[1,1].plot(pos,[np.mean(df_ventral_off['OOi_center_ctrl'].values),np.mean(df_ventral_off['OOi_center_drug'].values)], '-', color='k')
ax[3,1].scatter(pos,[np.mean(df_ventral_off['OOi_ring_ctrl'].values),np.mean(df_ventral_off['OOi_ring_drug'].values)], color='k')
ax[3,1].plot(pos,[np.mean(df_ventral_off['OOi_ring_ctrl'].values),np.mean(df_ventral_off['OOi_ring_drug'].values)], '-', color='k')
ax[5,1].scatter(pos,[np.mean(df_ventral_off['OOi_surround_ctrl'].values),np.mean(df_ventral_off['OOi_surround_drug'].values)], color='k')
ax[5,1].plot(pos,[np.mean(df_ventral_off['OOi_surround_ctrl'].values),np.mean(df_ventral_off['OOi_surround_drug'].values)], '-', color='k')


#SC_dorsal_on
for i in range(df_dorsal_on.shape[0]):
    ax[0,2].scatter(pos,[df_dorsal_on['SC_center_ctrl'].values[i],df_dorsal_on['SC_center_drug'].values[i]], color='darkred',alpha=0.05)
    ax[0,2].plot(pos,[df_dorsal_on['SC_center_ctrl'].values[i],df_dorsal_on['SC_center_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[2,2].scatter(pos,[df_dorsal_on['SC_ring_ctrl'].values[i],df_dorsal_on['SC_ring_drug'].values[i]], color='darkred',alpha=0.05)
    ax[2,2].plot(pos,[df_dorsal_on['SC_ring_ctrl'].values[i],df_dorsal_on['SC_ring_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[4,2].scatter(pos,[df_dorsal_on['SC_surround_ctrl'].values[i],df_dorsal_on['SC_surround_drug'].values[i]], color='darkred',alpha=0.05)
    ax[4,2].plot(pos,[df_dorsal_on['SC_surround_ctrl'].values[i],df_dorsal_on['SC_surround_drug'].values[i]], '-', color='darkred',alpha=0.05)
ax[0,2].scatter(pos,[np.mean(df_dorsal_on['SC_center_ctrl'].values),np.mean(df_dorsal_on['SC_center_drug'].values)], color='k')
ax[0,2].plot(pos,[np.mean(df_dorsal_on['SC_center_ctrl'].values),np.mean(df_dorsal_on['SC_center_drug'].values)], '-', color='k')
ax[2,2].scatter(pos,[np.mean(df_dorsal_on['SC_ring_ctrl'].values),np.mean(df_dorsal_on['SC_ring_drug'].values)], color='k')
ax[2,2].plot(pos,[np.mean(df_dorsal_on['SC_ring_ctrl'].values),np.mean(df_dorsal_on['SC_ring_drug'].values)], '-', color='k')
ax[4,2].scatter(pos,[np.mean(df_dorsal_on['SC_surround_ctrl'].values),np.mean(df_dorsal_on['SC_surround_drug'].values)], color='k')
ax[4,2].plot(pos,[np.mean(df_dorsal_on['SC_surround_ctrl'].values),np.mean(df_dorsal_on['SC_surround_drug'].values)], '-', color='k')
#SC_ventral_on
for i in range(df_ventral_on.shape[0]):
    ax[1,2].scatter(pos,[df_ventral_on['SC_center_ctrl'].values[i],df_ventral_on['SC_center_drug'].values[i]], color='darkred',alpha=0.05)
    ax[1,2].plot(pos,[df_ventral_on['SC_center_ctrl'].values[i],df_ventral_on['SC_center_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[3,2].scatter(pos,[df_ventral_on['SC_ring_ctrl'].values[i],df_ventral_on['SC_ring_drug'].values[i]], color='darkred',alpha=0.05)
    ax[3,2].plot(pos,[df_ventral_on['SC_ring_ctrl'].values[i],df_ventral_on['SC_ring_drug'].values[i]], '-', color='darkred',alpha=0.05)
    ax[5,2].scatter(pos,[df_ventral_on['SC_surround_ctrl'].values[i],df_ventral_on['SC_surround_drug'].values[i]], color='darkred',alpha=0.05)
    ax[5,2].plot(pos,[df_ventral_on['SC_surround_ctrl'].values[i],df_ventral_on['SC_surround_drug'].values[i]], '-', color='darkred',alpha=0.05)
ax[1,2].scatter(pos,[np.mean(df_ventral_on['SC_center_ctrl'].values),np.mean(df_ventral_on['SC_center_drug'].values)], color='k')
ax[1,2].plot(pos,[np.mean(df_ventral_on['SC_center_ctrl'].values),np.mean(df_ventral_on['SC_center_drug'].values)], '-', color='k')
ax[3,2].scatter(pos,[np.mean(df_ventral_on['SC_ring_ctrl'].values),np.mean(df_ventral_on['SC_ring_drug'].values)], color='k')
ax[3,2].plot(pos,[np.mean(df_ventral_on['SC_ring_ctrl'].values),np.mean(df_ventral_on['SC_ring_drug'].values)], '-', color='k',alpha=1)
ax[5,2].scatter(pos,[np.mean(df_ventral_on['SC_surround_ctrl'].values),np.mean(df_ventral_on['SC_surround_drug'].values)], color='k')
ax[5,2].plot(pos,[np.mean(df_ventral_on['SC_surround_ctrl'].values),np.mean(df_ventral_on['SC_surround_drug'].values)], '-', color='k')
# SC_dorsal_off
for i in range(df_dorsal_off.shape[0]):
    ax[0,3].scatter(pos,[df_dorsal_off['SC_center_ctrl'].values[i],df_dorsal_off['SC_center_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[0,3].plot(pos,[df_dorsal_off['SC_center_ctrl'].values[i],df_dorsal_off['SC_center_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[2,3].scatter(pos,[df_dorsal_off['SC_ring_ctrl'].values[i],df_dorsal_off['SC_ring_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[2,3].plot(pos,[df_dorsal_off['SC_ring_ctrl'].values[i],df_dorsal_off['SC_ring_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[4,3].scatter(pos,[df_dorsal_off['SC_surround_ctrl'].values[i],df_dorsal_off['SC_surround_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[4,3].plot(pos,[df_dorsal_off['SC_surround_ctrl'].values[i],df_dorsal_off['SC_surround_drug'].values[i]], '-', color='darkblue',alpha=0.05)
ax[0,3].scatter(pos,[np.mean(df_dorsal_off['SC_center_ctrl'].values),np.mean(df_dorsal_off['SC_center_drug'].values)], color='k')
ax[0,3].plot(pos,[np.mean(df_dorsal_off['SC_center_ctrl'].values),np.mean(df_dorsal_off['SC_center_drug'].values)], '-', color='k')
ax[2,3].scatter(pos,[np.mean(df_dorsal_off['SC_ring_ctrl'].values),np.mean(df_dorsal_off['SC_ring_drug'].values)], color='k')
ax[2,3].plot(pos,[np.mean(df_dorsal_off['SC_ring_ctrl'].values),np.mean(df_dorsal_off['SC_ring_drug'].values)], '-', color='k')
ax[4,3].scatter(pos,[np.mean(df_dorsal_off['SC_surround_ctrl'].values),np.mean(df_dorsal_off['SC_surround_drug'].values)], color='k')
ax[4,3].plot(pos,[np.mean(df_dorsal_off['SC_surround_ctrl'].values),np.mean(df_dorsal_off['SC_surround_drug'].values)], '-', color='k')
#SC_ventral_off
for i in range(df_ventral_off.shape[0]):
    ax[1,3].scatter(pos,[df_ventral_off['SC_center_ctrl'].values[i],df_ventral_off['SC_center_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[1,3].plot(pos,[df_ventral_off['SC_center_ctrl'].values[i],df_ventral_off['SC_center_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[3,3].scatter(pos,[df_ventral_off['SC_ring_ctrl'].values[i],df_ventral_off['SC_ring_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[3,3].plot(pos,[df_ventral_off['SC_ring_ctrl'].values[i],df_ventral_off['SC_ring_drug'].values[i]], '-', color='darkblue',alpha=0.05)
    ax[5,3].scatter(pos,[df_ventral_off['SC_surround_ctrl'].values[i],df_ventral_off['SC_surround_drug'].values[i]], color='darkblue',alpha=0.05)
    ax[5,3].plot(pos,[df_ventral_off['SC_surround_ctrl'].values[i],df_ventral_off['SC_surround_drug'].values[i]], '-', color='darkblue',alpha=0.05)
ax[1,3].scatter(pos,[np.mean(df_ventral_off['SC_center_ctrl'].values),np.mean(df_ventral_off['SC_center_drug'].values)], color='k')
ax[1,3].plot(pos,[np.mean(df_ventral_off['SC_center_ctrl'].values),np.mean(df_ventral_off['SC_center_drug'].values)], '-', color='k')
ax[3,3].scatter(pos,[np.mean(df_ventral_off['SC_ring_ctrl'].values),np.mean(df_ventral_off['SC_ring_drug'].values)], color='k')
ax[3,3].plot(pos,[np.mean(df_ventral_off['SC_ring_ctrl'].values),np.mean(df_ventral_off['SC_ring_drug'].values)], '-', color='k')
ax[5,3].scatter(pos,[np.mean(df_ventral_off['SC_surround_ctrl'].values),np.mean(df_ventral_off['SC_surround_drug'].values)], color='k')
ax[5,3].plot(pos,[np.mean(df_ventral_off['SC_surround_ctrl'].values),np.mean(df_ventral_off['SC_surround_drug'].values)], '-', color='k')

for current_row in range(6):
    for current_column in range(4):
        my_ax = ax[current_row,current_column] 
        my_ax.get_xaxis().set_tick_params(direction='out')
        my_ax.xaxis.set_ticks_position('bottom')
        my_ax.set_xticks(np.arange(1, len(label) + 1))
        my_ax.set_xticklabels(label,fontsize=12)
        my_ax.set_xlim(0.8, len(label) + 0.2)
        my_ax.axhline(0, color='gray', linestyle='dashed')
        my_ax.set_ylim(-1,1)
        
ax[0,0].set_title('On')
ax[0,1].set_title('Off')
ax[0,2].set_title('On')
ax[0,3].set_title('Off')
ax[5,0].set_xlabel('OOi')
ax[5,1].set_xlabel('OOi')
ax[5,2].set_xlabel('SC')
ax[5,3].set_xlabel('SC')
ax[0,0].text(0.2,0,'Center DORSAL', size=12,verticalalignment='center',rotation='90')
ax[1,0].text(0.2,0,'Center VENTRAL', size=12,verticalalignment='center',rotation='90')

ax[2,0].text(0.2,0,'Ring DORSAL', size=12,verticalalignment='center',rotation='90')
ax[3,0].text(0.2,0,'Ring VENTRAL', size=12,verticalalignment='center',rotation='90')

ax[4,0].text(0.2,0,'Surround DORSAL', size=12,verticalalignment='center',rotation='90')
ax[5,0].text(0.2,0,'Surround VENTRAL', size=12,verticalalignment='center',rotation='90')

sns.despine()
plt.show()

### Statistics

In [ ]:
#make uncorrected p-values array
from scipy import stats
p_vals = []

#SC
#TPMPA
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_center_ctrl'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine'], df_dorsal_on['SC_center_drug'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_center_ctrl'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine'], df_dorsal_off['SC_center_drug'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_center_ctrl'].loc[df_ventral_on['treatment']=='TPMPA;gabazine'], df_ventral_on['SC_center_drug'].loc[df_ventral_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_center_ctrl'].loc[df_ventral_off['treatment']=='TPMPA;gabazine'], df_ventral_off['SC_center_drug'].loc[df_ventral_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_ring_ctrl'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine'], df_dorsal_on['SC_ring_drug'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_ring_ctrl'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine'], df_dorsal_off['SC_ring_drug'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_ring_ctrl'].loc[df_ventral_on['treatment']=='TPMPA;gabazine'], df_ventral_on['SC_ring_drug'].loc[df_ventral_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_ring_ctrl'].loc[df_ventral_off['treatment']=='TPMPA;gabazine'], df_ventral_off['SC_ring_drug'].loc[df_ventral_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_surround_ctrl'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine'], df_dorsal_on['SC_surround_drug'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_surround_ctrl'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine'], df_dorsal_off['SC_surround_drug'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_surround_ctrl'].loc[df_ventral_on['treatment']=='TPMPA;gabazine'], df_ventral_on['SC_surround_drug'].loc[df_ventral_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_surround_ctrl'].loc[df_ventral_off['treatment']=='TPMPA;gabazine'], df_ventral_off['SC_surround_drug'].loc[df_ventral_off['treatment']=='TPMPA;gabazine']).pvalue)
#strychnine
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_center_ctrl'].loc[df_dorsal_on['treatment']=='strychnine'], df_dorsal_on['SC_center_drug'].loc[df_dorsal_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_center_ctrl'].loc[df_dorsal_off['treatment']=='strychnine'], df_dorsal_off['SC_center_drug'].loc[df_dorsal_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_center_ctrl'].loc[df_ventral_on['treatment']=='strychnine'], df_ventral_on['SC_center_drug'].loc[df_ventral_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_center_ctrl'].loc[df_ventral_off['treatment']=='strychnine'], df_ventral_off['SC_center_drug'].loc[df_ventral_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_ring_ctrl'].loc[df_dorsal_on['treatment']=='strychnine'], df_dorsal_on['SC_ring_drug'].loc[df_dorsal_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_ring_ctrl'].loc[df_dorsal_off['treatment']=='strychnine'], df_dorsal_off['SC_ring_drug'].loc[df_dorsal_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_ring_ctrl'].loc[df_ventral_on['treatment']=='strychnine'], df_ventral_on['SC_ring_drug'].loc[df_ventral_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_ring_ctrl'].loc[df_ventral_off['treatment']=='strychnine'], df_ventral_off['SC_ring_drug'].loc[df_ventral_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_surround_ctrl'].loc[df_dorsal_on['treatment']=='strychnine'], df_dorsal_on['SC_surround_drug'].loc[df_dorsal_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_surround_ctrl'].loc[df_dorsal_off['treatment']=='strychnine'], df_dorsal_off['SC_surround_drug'].loc[df_dorsal_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_surround_ctrl'].loc[df_ventral_on['treatment']=='strychnine'], df_ventral_on['SC_surround_drug'].loc[df_ventral_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_surround_ctrl'].loc[df_ventral_off['treatment']=='strychnine'], df_ventral_off['SC_surround_drug'].loc[df_ventral_off['treatment']=='strychnine']).pvalue)
#L-AP4
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_center_ctrl'].loc[df_dorsal_on['treatment']=='LAP4'], df_dorsal_on['SC_center_drug'].loc[df_dorsal_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_center_ctrl'].loc[df_dorsal_off['treatment']=='LAP4'], df_dorsal_off['SC_center_drug'].loc[df_dorsal_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_center_ctrl'].loc[df_ventral_on['treatment']=='LAP4'], df_ventral_on['SC_center_drug'].loc[df_ventral_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_center_ctrl'].loc[df_ventral_off['treatment']=='LAP4'], df_ventral_off['SC_center_drug'].loc[df_ventral_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_ring_ctrl'].loc[df_dorsal_on['treatment']=='LAP4'], df_dorsal_on['SC_ring_drug'].loc[df_dorsal_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_ring_ctrl'].loc[df_dorsal_off['treatment']=='LAP4'], df_dorsal_off['SC_ring_drug'].loc[df_dorsal_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_ring_ctrl'].loc[df_ventral_on['treatment']=='LAP4'], df_ventral_on['SC_ring_drug'].loc[df_ventral_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_ring_ctrl'].loc[df_ventral_off['treatment']=='LAP4'], df_ventral_off['SC_ring_drug'].loc[df_ventral_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_surround_ctrl'].loc[df_dorsal_on['treatment']=='LAP4'], df_dorsal_on['SC_surround_drug'].loc[df_dorsal_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_surround_ctrl'].loc[df_dorsal_off['treatment']=='LAP4'], df_dorsal_off['SC_surround_drug'].loc[df_dorsal_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_surround_ctrl'].loc[df_ventral_on['treatment']=='LAP4'], df_ventral_on['SC_surround_drug'].loc[df_ventral_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_surround_ctrl'].loc[df_ventral_off['treatment']=='LAP4'], df_ventral_off['SC_surround_drug'].loc[df_ventral_off['treatment']=='LAP4']).pvalue)
#UBP 310
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_center_ctrl'].loc[df_dorsal_on['treatment']=='UBP310'], df_dorsal_on['SC_center_drug'].loc[df_dorsal_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_center_ctrl'].loc[df_dorsal_off['treatment']=='UBP310'], df_dorsal_off['SC_center_drug'].loc[df_dorsal_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_center_ctrl'].loc[df_ventral_on['treatment']=='UBP310'], df_ventral_on['SC_center_drug'].loc[df_ventral_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_center_ctrl'].loc[df_ventral_off['treatment']=='UBP310'], df_ventral_off['SC_center_drug'].loc[df_ventral_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_ring_ctrl'].loc[df_dorsal_on['treatment']=='UBP310'], df_dorsal_on['SC_ring_drug'].loc[df_dorsal_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_ring_ctrl'].loc[df_dorsal_off['treatment']=='UBP310'], df_dorsal_off['SC_ring_drug'].loc[df_dorsal_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_ring_ctrl'].loc[df_ventral_on['treatment']=='UBP310'], df_ventral_on['SC_ring_drug'].loc[df_ventral_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_ring_ctrl'].loc[df_ventral_off['treatment']=='UBP310'], df_ventral_off['SC_ring_drug'].loc[df_ventral_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['SC_surround_ctrl'].loc[df_dorsal_on['treatment']=='UBP310'], df_dorsal_on['SC_surround_drug'].loc[df_dorsal_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['SC_surround_ctrl'].loc[df_dorsal_off['treatment']=='UBP310'], df_dorsal_off['SC_surround_drug'].loc[df_dorsal_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['SC_surround_ctrl'].loc[df_ventral_on['treatment']=='UBP310'], df_ventral_on['SC_surround_drug'].loc[df_ventral_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['SC_surround_ctrl'].loc[df_ventral_off['treatment']=='UBP310'], df_ventral_off['SC_surround_drug'].loc[df_ventral_off['treatment']=='UBP310']).pvalue)

#OOi
#TPMPA
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_center_ctrl'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine'], df_dorsal_on['OOi_center_drug'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_center_ctrl'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine'], df_dorsal_off['OOi_center_drug'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_center_ctrl'].loc[df_ventral_on['treatment']=='TPMPA;gabazine'], df_ventral_on['OOi_center_drug'].loc[df_ventral_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_center_ctrl'].loc[df_ventral_off['treatment']=='TPMPA;gabazine'], df_ventral_off['OOi_center_drug'].loc[df_ventral_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_ring_ctrl'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine'], df_dorsal_on['OOi_ring_drug'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_ring_ctrl'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine'], df_dorsal_off['OOi_ring_drug'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_ring_ctrl'].loc[df_ventral_on['treatment']=='TPMPA;gabazine'], df_ventral_on['OOi_ring_drug'].loc[df_ventral_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_ring_ctrl'].loc[df_ventral_off['treatment']=='TPMPA;gabazine'], df_ventral_off['OOi_ring_drug'].loc[df_ventral_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_surround_ctrl'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine'], df_dorsal_on['OOi_surround_drug'].loc[df_dorsal_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_surround_ctrl'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine'], df_dorsal_off['OOi_surround_drug'].loc[df_dorsal_off['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_surround_ctrl'].loc[df_ventral_on['treatment']=='TPMPA;gabazine'], df_ventral_on['OOi_surround_drug'].loc[df_ventral_on['treatment']=='TPMPA;gabazine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_surround_ctrl'].loc[df_ventral_off['treatment']=='TPMPA;gabazine'], df_ventral_off['OOi_surround_drug'].loc[df_ventral_off['treatment']=='TPMPA;gabazine']).pvalue)

#strychnine
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_center_ctrl'].loc[df_dorsal_on['treatment']=='strychnine'], df_dorsal_on['OOi_center_drug'].loc[df_dorsal_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_center_ctrl'].loc[df_dorsal_off['treatment']=='strychnine'], df_dorsal_off['OOi_center_drug'].loc[df_dorsal_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_center_ctrl'].loc[df_ventral_on['treatment']=='strychnine'], df_ventral_on['OOi_center_drug'].loc[df_ventral_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_center_ctrl'].loc[df_ventral_off['treatment']=='strychnine'], df_ventral_off['OOi_center_drug'].loc[df_ventral_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_ring_ctrl'].loc[df_dorsal_on['treatment']=='strychnine'], df_dorsal_on['OOi_ring_drug'].loc[df_dorsal_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_ring_ctrl'].loc[df_dorsal_off['treatment']=='strychnine'], df_dorsal_off['OOi_ring_drug'].loc[df_dorsal_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_ring_ctrl'].loc[df_ventral_on['treatment']=='strychnine'], df_ventral_on['OOi_ring_drug'].loc[df_ventral_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_ring_ctrl'].loc[df_ventral_off['treatment']=='strychnine'], df_ventral_off['OOi_ring_drug'].loc[df_ventral_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_surround_ctrl'].loc[df_dorsal_on['treatment']=='strychnine'], df_dorsal_on['OOi_surround_drug'].loc[df_dorsal_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_surround_ctrl'].loc[df_dorsal_off['treatment']=='strychnine'], df_dorsal_off['OOi_surround_drug'].loc[df_dorsal_off['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_surround_ctrl'].loc[df_ventral_on['treatment']=='strychnine'], df_ventral_on['OOi_surround_drug'].loc[df_ventral_on['treatment']=='strychnine']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_surround_ctrl'].loc[df_ventral_off['treatment']=='strychnine'], df_ventral_off['OOi_surround_drug'].loc[df_ventral_off['treatment']=='strychnine']).pvalue)

#L-AP4
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_center_ctrl'].loc[df_dorsal_on['treatment']=='LAP4'], df_dorsal_on['OOi_center_drug'].loc[df_dorsal_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_center_ctrl'].loc[df_dorsal_off['treatment']=='LAP4'], df_dorsal_off['OOi_center_drug'].loc[df_dorsal_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_center_ctrl'].loc[df_ventral_on['treatment']=='LAP4'], df_ventral_on['OOi_center_drug'].loc[df_ventral_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_center_ctrl'].loc[df_ventral_off['treatment']=='LAP4'], df_ventral_off['OOi_center_drug'].loc[df_ventral_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_ring_ctrl'].loc[df_dorsal_on['treatment']=='LAP4'], df_dorsal_on['OOi_ring_drug'].loc[df_dorsal_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_ring_ctrl'].loc[df_dorsal_off['treatment']=='LAP4'], df_dorsal_off['OOi_ring_drug'].loc[df_dorsal_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_ring_ctrl'].loc[df_ventral_on['treatment']=='LAP4'], df_ventral_on['OOi_ring_drug'].loc[df_ventral_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_ring_ctrl'].loc[df_ventral_off['treatment']=='LAP4'], df_ventral_off['OOi_ring_drug'].loc[df_ventral_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_surround_ctrl'].loc[df_dorsal_on['treatment']=='LAP4'], df_dorsal_on['OOi_surround_drug'].loc[df_dorsal_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_surround_ctrl'].loc[df_dorsal_off['treatment']=='LAP4'], df_dorsal_off['OOi_surround_drug'].loc[df_dorsal_off['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_surround_ctrl'].loc[df_ventral_on['treatment']=='LAP4'], df_ventral_on['OOi_surround_drug'].loc[df_ventral_on['treatment']=='LAP4']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_surround_ctrl'].loc[df_ventral_off['treatment']=='LAP4'], df_ventral_off['OOi_surround_drug'].loc[df_ventral_off['treatment']=='LAP4']).pvalue)

#UBP 310
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_center_ctrl'].loc[df_dorsal_on['treatment']=='UBP310'], df_dorsal_on['OOi_center_drug'].loc[df_dorsal_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_center_ctrl'].loc[df_dorsal_off['treatment']=='UBP310'], df_dorsal_off['OOi_center_drug'].loc[df_dorsal_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_center_ctrl'].loc[df_ventral_on['treatment']=='UBP310'], df_ventral_on['OOi_center_drug'].loc[df_ventral_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_center_ctrl'].loc[df_ventral_off['treatment']=='UBP310'], df_ventral_off['OOi_center_drug'].loc[df_ventral_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_ring_ctrl'].loc[df_dorsal_on['treatment']=='UBP310'], df_dorsal_on['OOi_ring_drug'].loc[df_dorsal_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_ring_ctrl'].loc[df_dorsal_off['treatment']=='UBP310'], df_dorsal_off['OOi_ring_drug'].loc[df_dorsal_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_ring_ctrl'].loc[df_ventral_on['treatment']=='UBP310'], df_ventral_on['OOi_ring_drug'].loc[df_ventral_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_ring_ctrl'].loc[df_ventral_off['treatment']=='UBP310'], df_ventral_off['OOi_ring_drug'].loc[df_ventral_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_on['OOi_surround_ctrl'].loc[df_dorsal_on['treatment']=='UBP310'], df_dorsal_on['OOi_surround_drug'].loc[df_dorsal_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_dorsal_off['OOi_surround_ctrl'].loc[df_dorsal_off['treatment']=='UBP310'], df_dorsal_off['OOi_surround_drug'].loc[df_dorsal_off['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_on['OOi_surround_ctrl'].loc[df_ventral_on['treatment']=='UBP310'], df_ventral_on['OOi_surround_drug'].loc[df_ventral_on['treatment']=='UBP310']).pvalue)
p_vals.append(stats.ttest_rel(df_ventral_off['OOi_surround_ctrl'].loc[df_ventral_off['treatment']=='UBP310'], df_ventral_off['OOi_surround_drug'].loc[df_ventral_off['treatment']=='UBP310']).pvalue)

In [ ]:
import statsmodels.stats.multitest

reject, p_corr, _, alpha_corr = statsmodels.stats.multitest.multipletests(p_vals, alpha=0.05, method='bonferroni')

In [ ]:
# list
drug = ['TPMPA;','strychnine;','LAP4;','UBP310;']
polarity = ['ON', 'OFF']
space = ['Centre;', 'Ring;', 'Surround;']
retina = ['Dorsal;', 'Ventral;']
order_tests = []

# SC
for drug_i in drug:
    for space_i in space:
        for retina_i in retina:
            for polarity_i in polarity:
                order_tests.append('SC: ' + drug_i +' ' + space_i +' ' + retina_i +' ' + polarity_i)
# OOi
for drug_i in drug:
    for space_i in space:
        for retina_i in retina:
            for polarity_i in polarity:
                order_tests.append('OOi: ' + drug_i +' ' + space_i +' ' + retina_i +' ' + polarity_i)


np.array(order_tests)[(np.where(reject)[0])]